# Assessing LLMs Suitability for Knowledge Graph Completion

!!! You need your own API Keys for Hugging Face and OpenAI. The notebook is intended to work on Google Colab, therefore save them there as Secrets. Save the HF token as "HF_TOKEN", and the OpenAI token as "GPT_TOKEN". The code will load them automatically, if instructions are followed.

!!! As of 30/05/2025, HuggingFace imposed a small limit to the number of inference API calls. In our case, we could only send 26 calls to Mixtral-8x7b until the limit for a month was hit. Therefore, any model from HF needs to be locally loaded. We now provide this option in code, for Mixtral-8x7b, with a new function called *query2*. Keep in mind that some models necessitate huge amounts of GPU RAM, so choose an appropiate Colab configuration (in our case, GPU A100).

Recent studies have demonstrated that Large Language Models (LLMs) can perform various Knowledge Graph-related tasks, including Knowledge Graph Construction, even in Zero- and Few-Shot settings. However, LLMs are prone to hallucinating information and producing non-deterministic outputs, which can result in flawed reasoning, even when the answers appear to meet user expectations. This unpredictability limits their integration into automated natural language processing pipelines, such as those used in chatbots or Task-Oriented Dialogue systems. To explore the potential and limitations of LLMs in Knowledge Graph tasks, we evaluate three prominent models, namely Mixtral-8x7b-Instruct-v0.1, GPT-3.5-Turbo-0125, and GPT-4o, on constructing static knowledge graphs. Our approach uses prompts based on the TELeR taxonomy in Zero- and One-Shot sce- narios, within the context of a Task-Oriented Dialogue system. We also propose a flexible evaluation framework that captures all usable information generated by the models, alongside traditional strict metrics, and introduce TODSet, a dataset tailored to gauge the performance of LLMs on knowledge graph-related tasks. Our findings suggest that, with well-designed prompts containing sufficient detail and examples, LLMs can effectively contribute to Knowledge Graph Construction tasks.

**UPDATE (30/05/2025)**: The two datasets, Templates Easy and Templates Hard, are now referred to as one, namely TODSet. Still, each of them have a different set of samples, thus the code and experiments do not change.

**UPDATE (30/05/2025)**: We now introduce experiments on a new dataset, called FootballSet. To evaluate our methodology in a more complex setting, we adapted the DBpedia-WebNLG sports ontology and dataset, originally developed by Mihindukulasooriya et al. (https://github.com/cenguix/Text2KGBench), to align with our format. This adaptation, inspired by their work, enabled us to assess model performance under increased ontological complexity.

All the necessary resources to reproduce the experiments are at -> https://github.com/IonutIga/LLMs-for-KGC

For TODSet:
To generate the datasets from scratch, simply load either *templates_easy.txt* or *templates_hard.txt*, in the "Generate dataset" sub-section.

For TODSet: To load the datasets, simply load either *templates_easy_final.txt* or *templates_hard_final.txt*, in the "Load dataset" sub-section.

For FootballSet: To load the dataset, simply load *football_dataset.txt*, in the "Load dataset" sub-section.

To generate the prompts from scratch, you have to first generate/load a dataset, then enter the "Generate prompts" sub-section, where you have to choose the type of prompts to be attached to the input (either system_prompts or final_model_prompts).

To load the prompts, simply load the desired file in the "Load prompts" sub-section.

To run the experiments on a given prompt, simply run the "General pipeline" sub-section. Enter it to set variables according to your needs.

To measure already existing runs (even the ones provided by the paper), enter the "Calculate metrics from existing predictions" sub-section, load the desired predictions either from the hub (files with "run_..." as name) or generated b yourself and set at which levels you wish to generate metrics.

## Install and import libraries

In [ ]:
!pip install --upgrade openai # GPT library
!pip install --upgrade tiktoken # count tokens in an input for openai
!pip install transformers # for all other models available on HuggingFace
!pip install pydantic # data validation
!pip install --upgrade bitsandbytes
# !pip install python-dotenv # load env variables
# !pip install --upgrade accelerate bitsandbytes # GPU optimization and quantization

In [2]:
from openai import OpenAI
import tiktoken
import os
from transformers import  AutoTokenizer # to load the correct tokenizers of specific models
import requests # to send requests to HG's serveless API
import torch # multi purposes
from google.colab import userdata # to load the secret keys
import gc
from datetime import datetime # for the current date and time
import json
from tqdm import tqdm # to display the loading time
from google.colab import files # for files workloads
import re # for regex
from jinja2.exceptions import TemplateError # for the apply chat_templates function
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, pipeline
# might be useful for alternatives in code
#from itertools import combinations
# from dotenv import load_dotenv
# from transformers import pipeline
# from transformers import AutoModelForCausalLM
# from huggingface_hub import InferenceClient

## Define functions and classes

In [3]:
# class to hold information regarding each prompt fed to a model


class Prompt:

  '''
  Parameters
  ----------
    prompt : str
            The system prompt
    text : str
          The input text
    golden_labels : list
                    The list of correct triples to be predicted
    alternative_labels: list
                        The list of possbile alternative triples to some golden ones; Under flexible metrics, it will be counted as correct;
                        eg. For :Project1 :hasManager :Employee1; :Employee1 rdf:type :Employee; :Employee1 :hasName "X"; the triple :Project1 :hasManager "X" is accepted;
                        The final metrics won't subsitute whole three triples with one, but count it as a correct alternative with a 1/3 ratio
    fp_ok_labels : list
                  "false positive okay labels", the list of predictions that may be accepted as okay, but are not part of the golden labels;
                    False positives that might have been counted as corrected if additional background knowledge was provided;
                    eg. When the triple :Employee1 :hasRole "manager" is generated, as the model inferred it from the :Project1 :hasManager :Employee1;
                    The final metrics will not count them as correct, but reduce the number of false positives
    version : float
              The version of the system prompt
    level  : float
            The level of the system prompt, on a scale of 1, 2, 3, 4.1, and 4.2
    text_type : str
                The type of the input text, as mentioned in the provided templates; the templates file come with a legend
    class_type : str
                The type of the instance mentione in the input text, from the ontology; as for now, Project, Employee, Status, or None if the type is not in the ontology
    sys_mesg_ord : int
                  The position of the system prompt; 0 - at the beginning of the final prompt, 1 - at the end of the final prompt
  '''


  def __init__(self, prompt, text, golden_labels, alternative_labels, fp_ok_labels, version, level, text_type, class_type, sys_mesg_ord = 0):

    self.prompt = prompt
    self.text = text
    self.metadata = []
    self.golden_labels = golden_labels
    self.alternative_labels = alternative_labels
    self.fp_ok_labels = fp_ok_labels
    self.version = version
    self.level = level
    self.text_type = text_type
    self.class_type = class_type
    self.data_index = -1 # the index of the data from the dataset for later references

    if sys_mesg_ord == 0:

      self.messages = [{'role': 'system', 'content' : self.prompt}, {'role': 'user', 'content' : f' Input text: {self.text}'}]
    else:
     self.messages = [{'role': 'user', 'content' : f' Input text: {self.text}'}, {'role': 'system', 'content' : f'\n {self.prompt}'}]

  def append_metadata(self, metadata):
    self.metadata.append(metadata)

  # function to convert the Prompt object to a dictionary, for serialization purposes

  '''
  Return
  ------
    type : dict
          The Prompt object converted in a dictionary
  '''

  def to_json(self):

    json_prompt = {
                    "prompt" : self.prompt,
                    "text" : self.text,
                    "metadata" : self.metadata,
                    "golden_labels" : self.golden_labels,
                    "alternative_labels" : self.alternative_labels,
                    "fp_ok_labels" : self.fp_ok_labels,
                    "version" : self.version,
                    "level" : self.level,
                    "text_type" : self.text_type,
                    "class_type" :self.class_type,
                    "data_index" : self.data_index
                    }

    return json_prompt

In [4]:
# function to read lines from a file

'''
Parameters
----------
  file : str
        The name of the file to be read
'''
'''
Return
------
  type : str
         The contained text of the file
'''

def read_lines(file):
  f = open(file)
  lines = f.read().splitlines()
  f.close()
  return lines

In [5]:
# function to count the tokens from the input message of a OpenAI's model
# copied from their documentation

'''
Parameters
  ----------
  messages : list
            The list containing dictionaries of the role (user/system) and associated message
  model : str
          The name of the OpenAI model
'''
'''
Return
------
  type : int
         The number of tokens
'''

def num_tokens_from_messages_openai(messages, model="gpt-3.5-turbo-0125"):

    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0125",
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0125.")
        return num_tokens_from_messages_openai(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages_openai(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [6]:
# function to count the tokens from the input message of HG models

'''
Parameters
  ----------
  tokenized_chat : str
            The text fed into the model, after applying its template design
  tokenizer : Tokenizer
          The tokenizer specific to the model
'''
'''
Return
------
  type : int
         The number of tokens
'''
def num_tokens_from_messages(tokenized_chat, tokenizer):
  return len(tokenizer.encode(tokenized_chat))

In [7]:
# function to view the predictions of the models from the checkpoint
# it takes as input a list of dictionaries that contain other lists of Prompt objects
# it loops through each prompt while prompting useful information from it

'''
Parameters
  ----------
  predictions : list
                The list of dictionaries that contain other lists of Prompt objects which hold the predictions of each model
  all_prompts : list
                The list of dictionaries that contain other lists of Prompt objects in their initial state; here used only to provide the initial number of Prompts to ensure that all predictions are generated
  show_system_prompt : bool
                Show the system prompt, which contains the directives to the model, without the input to work on
  generate_model_prompts : bool
                If set to true, the function will interpret each prediction as a new system prompt specific to a model and output a new set of prompts; eg. when we ask the models to reformulate the standard prompts
  model_prompts: dict
                If generate_model_prompts = True, and a dict is provided, the model will append to it the generated model-specific system prompts
'''

'''
Return
------
  If generate_model_prompts = True
    type : dict
           The dictionary containing the output prompts
  Else
    Prints the content of each Prompt object
'''

def view_predictions(predictions, all_prompts, show_system_prompt = True, generate_model_prompts = False, model_prompts = {}):

  for i, pred_per_model in enumerate(predictions):

    for model, pred_list in pred_per_model.items():
      print(f'Predictions: {len(pred_list)}/{len(all_prompts[i][model])}')

      for pred in pred_list:

        print('-' * 25, f'Data instance index {pred.data_index}, prompt version {pred.version} and level {pred.level}, class type {pred.class_type}, text type {pred.text_type}', '-' * 25, '\n')

        try:

          for metadata in pred.metadata: # each prediction of a model is hold as a dictionary in the metadata of a prompt

            version_dict = {}
            version_list = []
            level_text = {}
            flag = False

            print('*' * 25, f'Model: {metadata["model"]}', '*' * 25) if 'model' in metadata.keys() else print('*' * 25, f'Model: unknown', '*' * 25)

            if generate_model_prompts: # if the goal of the predictions was to reformulate/generate model-specific system prompts

              if metadata["model"] in model_prompts.keys():
                    version_dict = model_prompts[metadata["model"]]

              if pred.version in version_dict.keys():
                version_list = version_dict[pred.version] # override the existing system propmts, if at the same version

              for i,lt in enumerate(version_list):
                if pred.level in lt.keys():
                  flag = i
              if flag != False: # override the existing system prompts, if at same level
                version_list[flag][pred.level] = metadata['generated_text']
              else:
                version_list.append({pred.level : metadata['generated_text']})

              version_dict[pred.version] = version_list
              model_prompts[metadata["model"]] = version_dict


            if 'processing-ms' in metadata.keys() :print(f'\nTime to generate (s): {int(metadata["processing-ms"]) / 1000}')
            print(f'Number of tokens of the prompt: {metadata["tokens"]}')
            if show_system_prompt:
              print(f'Prompt: {metadata["prompt"]}')
            print(f'Generated text: {metadata["generated_text"]}')
            print(f'Alternative labels: {pred.alternative_labels}')
            print(f'FP accepted labels: {pred.fp_ok_labels}')
            print(f'Golden labels: {pred.golden_labels}\n')

        except Exception as e:
          print(e,"Error in parsing the Prompt object.")

  if generate_model_prompts:
    return model_prompts

In [8]:
# function to create prompts that indicate a reprhasing task; to obtain new, model-specific system prompts

'''
Parameters
----------
  system_prompts : list
                   List containing Prompt objects; should only be objects that have different levels and/or versions of the system prompt
'''
'''
Return
------
  type : list
         The list of prompts with the task of rephrasing the input text; the list has a dictionary structure similar to the one obtained via generating the standard prompts
'''

def rephrase_system_prompts(system_prompts):
  rephrase_prompt = 'You are given a test prompt delimited by double quotes. Your task is to rephrase the prompt in order for you to better understand it, without changing the specified output format and any provided examples. Output only the rephrased prompt in double quotes. \n Input prompt: '
  rr_prompts = [{'all' : []}]
  for prompt in system_prompts:
    p = Prompt(rephrase_prompt,f' "{prompt.prompt}"',['None'],['None'],['None'], prompt.version, prompt.level, 'Rephrase', 'None', 0) # no golden/alternative/fp_ok labels needed
    rr_prompts[0]['all'].append(p)
  return rr_prompts

In [9]:
# function to generate the Prompt objects

'''
Parameters
----------
  system_prompts : dict
                   The dictionary containing the system prompts of a model/all models, on versions and levels
  dataset : list
            The list containing the dictionaries with the input text, golden/alternative/fp_ok labels, text and class type
  versions : list
             The list with the versions of the system prompts to be included; 'all' is default for all versions
  levels : list
             The list with the levels of the system prompts to be included; 'all' is default for all levels
  sys_mesg_ord : int
                  The position of the system prompt; 0 - at the beginning of the final prompt, 1 - at the end of the final prompt
  file : str
         The name of the file to place the serialized Prompt objects for later reuse
  download : bool
             Option to automatically download the file
'''
'''
Return
------
  type : list
         The list of all generated prompts; alternatively, it generates a text file of the serialized Prompt objects
'''

def generate_prompts(system_prompts, dataset, versions = ['all'], levels = ['all'], sys_mesg_ord = 0, file = 'prompts.txt', download = True):

  all_prompts = []

  i = 0

  for model, versionss in system_prompts.items(): # loop through all system prompts specific to a models/all models' version(s)

    prompts_per_model = {}
    prompts_per_model[model] = []

    for version, prompts in versionss.items(): # loop through all prompts at a specific version

      if version in versions or versions[0] == 'all':
        for prompt in tqdm(prompts):
          for level, text in prompt.items(): # loop through prompts at certain levels
            for j, instance in enumerate(dataset): # loop through each dataset instance; the final Prompt objects will contain each instance at a certain level, followed by the next level and so on
              if level in levels or levels[0] == 'all':
                if level in [4.1, 4.2]: # this labels are few-shot ones, therefore they need an example to be attached to them
                  if instance['golden_labels'] == 'None': # attach appropiate example to the golden labels
                    p = Prompt(text + example_none if level == 4.1 else text + example_none_cot, instance['text'], instance['golden_labels'],instance['alternative_labels'],instance['fp_ok_labels'], version, level, instance['text_type'], instance['class_type'], sys_mesg_ord = sys_mesg_ord)
                  else:
                    p = Prompt(text + example_labels if level == 4.1 else text + example_labels_cot, instance['text'], instance['golden_labels'], instance['alternative_labels'],instance['fp_ok_labels'], version, level, instance['text_type'], instance['class_type'], sys_mesg_ord = sys_mesg_ord)
                else:
                  p = Prompt(text, instance['text'], instance['golden_labels'], instance['alternative_labels'],instance['fp_ok_labels'],  version, level, instance['text_type'], instance['class_type'],sys_mesg_ord = sys_mesg_ord)

              p.data_index = j
              prompts_per_model[model].append(p)
              write_prompt(p, file)
              i += 1

    all_prompts.append(prompts_per_model)

  if download:
    files.download(file)

  return all_prompts

In [10]:
# function to write a Prompt object as a text serialization

'''
Parameters
----------
  prompt : Prompt
           The Prompt object
  file : str
         The name of the final where to write the serialization
'''

def write_prompt(prompt, file = 'prompts.txt'):
  f = open(file, 'a')
  f.write(str(prompt.to_json()))
  f.write('\n')
  f.close()

In [11]:
# function to deserialize the text version of a Prompt object

'''
Parameters
----------
  file : str
         The file from which to read the text
'''
'''
Return
------
  type : list
         The list with the deserialized Prompt objects; similar format to the generate_prompts function
'''

def from_text_to_Prompt(file = 'prompts.txt'):
  all_prompts = []
  prompts = {'all' : []} # put them under the label of 'all', even if they were originally under a model's name; after all, it doesn't really matter as the Prompt object metadata contains the model's name under each prediction
  lines = read_lines(file)
  for line in tqdm(lines):
    prompt = eval(line) # eval it, generate a dictionary back
    new_prompt = Prompt(prompt['prompt'], prompt['text'], prompt['golden_labels'], prompt['alternative_labels'], prompt['fp_ok_labels'], prompt['version'], prompt['level'], prompt['text_type'], prompt['class_type'])
    new_prompt.metadata = prompt['metadata']
    new_prompt.data_index = prompt['data_index']
    prompts['all'].append(new_prompt)

  all_prompts.append(prompts)
  return all_prompts

In [12]:
# function to generate a dataset from a text file where serialized instances were output by generate_dataset function
# useful to not regenerate the dataset each time you run the experiments

'''
Parameters
----------
  file : str
         Name of the file from where to read the text
'''
'''
Return
------
  type : list
         The dataset, similar to generate_dataset function
'''

def from_text_to_dataset(file):
  dataset = []
  lines = read_lines(file)
  for line in tqdm(lines):
    instance = eval(line)
    dataset.append(instance)
  return dataset

In [13]:
# function to generate a dataset from a template text file (specific template style)

'''
Parameters
----------
  file : str
         Path of the file that stores the templates to be read
  dataset : list
            If a dataset is already generated and one wishes to add to it, they can provide an already existing list
  download : bool
             Option to download the text file with the generated dataset
'''
'''
Return
------
  type : list
         The list with the instances containing the input text, golden/alternative/fp_ok labels, text and class type
'''

def generate_dataset(file = "", dataset = [], download = True):

  lines = read_lines(file)
  dataset_file_path = f"{file[:-4]}_final.txt"

  for line in tqdm(lines):

    instance = {}
    part_lines = line.split("*") # each line has input_text*relationships*objects*intent*text_type

    instance["text"] = f"{part_lines[0]}."
    golden_labels = []
    alternative_labels = []
    fp_ok_labels = []
    instance_type = ""

    if part_lines[3] == "insert": # for the moment, only the texts with the intent type of 'insert' are accepted
      for k, v in zip(part_lines[1].split("|"), part_lines[2].split("|")): # go in parallel over relationship-object pairs

        if k == "entity": # if entity, create the appropiate id as 'Class1'
          instance_type = v.capitalize()
          id = instance_type + "1" # a simple way to create an id not to confuse the models, as for now

          if instance_type in ["Project", "Status", "Employee"]: # only accept the supported classes from the ontology, else is None
            golden_labels.append({"subject" : f"{id}", "relationship" : "rdf:type", "object" : instance_type})
          else:
            golden_labels = "None"
            break

        elif instance_type == "Project" and k in ["hasManager","hasStatus"]: # these relationships might refer to other instances

          sub_instance_type = "Employee" if k == "hasManager" else "Status"
          sub_id = v.split(".")[1] if v.startswith("id.") else sub_instance_type + "1" # id follows the same format as above, if not provided with the template

          if v.startswith("role.") == False:

            if v.startswith("id.") == False: # if no id is provided by the template, we can assume the text refers to the employee's name
              golden_labels.append({"subject" : f"{sub_id}", "relationship" : "hasName", "object" : v})
              alternative_labels.append({"subject" : f"{id}", "relationship" : k, "object" : v}) # add as alternative label :Project1 :hasManager "name"

            if k == "hasManager": # then triples referring to their role as manager are fp_ok accepted
              fp_ok_labels.append({"subject" : f"{sub_id}", "relationship" : "hasRole", "object" : "Manager"})
              fp_ok_labels.append({"subject" : f"{sub_id}", "relationship" : "hasRole", "object" : "manager"})

          else: # when role is provided, it is appended to the golden labels
            golden_labels.append({"subject" : f"{sub_id}", "relationship" : "hasRole", "object" : v.split(".")[1]})

          golden_labels.append({"subject" : f"{id}", "relationship" : k, "object" : f"{sub_id}"}) # the direct relationship between instances
          golden_labels.append({"subject" : f"{sub_id}", "relationship" : "rdf:type", "object" : f"{sub_instance_type}"}) # the rdf:type relationship of the sub instance

        else: # any other relationship
          golden_labels.append({"subject" : f"{id}", "relationship" : k, "object" : v})

      instance["golden_labels"] = f'{golden_labels}' if type(golden_labels) is list else golden_labels # if None is the only golden label provided, then there is no list, just "None"
      instance["alternative_labels"] = f'{alternative_labels}'
      instance["fp_ok_labels"] = f'{fp_ok_labels}'
      instance["class_type"] = instance_type
      instance["text_type"] = part_lines[4]

      dataset.append(instance)

      f = open(dataset_file_path, "a") # write it in the text file
      f.write(str(instance))
      f.write("\n")
      f.close()

  if download:
    files.download(dataset_file_path)


  return dataset

In [14]:
# function to find the required json objects in the output of the models and to serialize provided golden/alternative/fp_ok labels to the same format with the output
# it allows two metrics measurements styles: strict and flexible;
# strict metrics needs the output to exactly follow the instructions given in the system prompts
# flexible metrics allows non-conforming outputs to still be allowed, with a penalization that can be applied to the accuracy metric;
# it assumes '1' is the max possible accuracy value, therefore: 0.1 max pen for not outputing in a list (only one) format; 0.01 for providing the full IRI of a value; 0.01 for adding ':' in front of a value; 0.1 if a triple doesn't follow the provided format, while also not adding it
# the strict paradigm allows for full verification of models respecting the input prompt, while flexible allows for some room where post-processing steps can convert the output in a correct format
# !!! GPT models have a tendency to surround their output with a JSON format tag (i.e. "```json ... ```"), due to their ability to output JSON-formatted output when requested; while on strict metrics, we want the output to be as requested (namely just a list of JSON objects). We have to do an extra processing step that we consider trivial, thus not count it as an error

'''
Parameters
----------
  text : str
         The text to be searched for json objects
  strict : bool
           True for the strict paradigm, False for the flexible one
'''
'''
Return
------
  type : (str/list, float)
         Return a tuple containing either 'er1ror'/'incorrect format'/list of json objects and a float number represting the penalty value
'''

def find_json_objects(text, strict = False):

  pattern = r'\{(?:[^{}]|)*\}' # Regular expression pattern to match JSON objects
  penalize = 0
  flag = False # flag to check whether the output json object respects the provided format

  text = text.strip()

  if text.startswith('```json'): # GPT models surround their output with a JSON format tag, as they can generate such text when requested; we process it out
    text = text.replace('```json','')

  text = text.strip()

  if text.endswith('```'): # as well as the end tag
    text = text.replace('```','')

  text = text.strip() # remove any trailing whitespaces

  if text.startswith('er1ror'): # custom text to mark an error from the server; written so it is not missunderstood as an actual correct text
    print(text)
    return 'error', penalize

  json_objects = re.findall(pattern, text) # find json objects; all of them from the output, even if the model rewrites some examples as in COT type prompts

  if not json_objects:

    if strict: # on strict, only "None" has to be the output

      if text == 'None':

        parsed_objects = ["None"]

      else:
        return 'incorrect format', penalize # else it is an incorrect format output

    else:
      parsed_objects = ["None"] # flexible allows for any output that does not contain json objects to be considered as "None"


  else: # Parse each JSON object and store them in a list

    list_starts = re.findall('\[',text)
    if len(list_starts) > 1: # the format requires only one list to be output
      if strict == False:
        penalize += 0.025
      else:
        return 'incorrect format', 0

    if text.startswith('[') == False: # the output must contain only the list
      if strict == False:
        penalize += 0.075
      else:
        return 'incorrect format', 0

    elif text.endswith(']') == False: # and finish as a list
      if strict == False:
        penalize += 0.075
      else:
        return 'incorrect format', 0

    parsed_objects = []

    for obj in json_objects:
        try:

            if strict: # no post-processing to be done

              try:

                eval_obj = eval(obj)

                if type(eval_obj) is dict: # allow only dict/json objects triples

                  if len(eval_obj.keys()) == 3: # only three keys are needed as for the format

                    for k, v in eval_obj.items():

                      if k.lower() in ['subject', 'relationship', 'object']: # only this keys are allowed

                        if type(v) is not list:

                          v = str(v)
                          if v.find(':') != -1:
                            iri, thing = v.split(":")[0],v.split(":")[1]
                            if iri:
                              v = f'{thing}'
                              eval_obj[k] = v


                parsed_objects.append(eval_obj)
              except:
                pass

            else:

              transformed_text = re.sub(r'(\S+)#(\S+)', '"' + r'\2', obj) # remove IRI with #, eg. http://www.example.org/john#Employee
              if transformed_text != obj:
                penalize += 0.01 # penalize each IRI removal

              transformed_text2 = re.sub(r'(\S+)/(\S+)', '"' + r'\2', transformed_text) # remove IRI with /, eg. http://www.example.org/john/Employee

              if transformed_text2 != transformed_text:
                penalize += 0.01 # penalize each IRI removal
              try:
                triple = eval(transformed_text2)
              except:
                triple = ''

              if type(triple) is dict: # allow only dict/json objects triples

                if len(triple.keys()) == 3: # only three keys are needed as for the format

                  for k, v in triple.items():
                    flag = False
                    if k.lower() in ['subject', 'relationship', 'object']: # only this keys are allowed

                      if type(v) is not list:

                        #print(v)
                        v = str(v)
                        if v.find(':') != -1:
                            iri, thing = v.split(":")[0],v.split(":")[1]
                            if iri:
                              v = f'{thing}'
                              triple[k] = v
                            else:
                              v = f':{thing}'
                        if type(v) == str and v.startswith(':'):
                          triple[k] = v[1:] # remove : from value texts, eg. 'subject' : ':Employee1'
                          penalize += 0.01 # penalize the removal
                        #print(v)
                    else:
                      flag = True
                      break
                else:
                  flag = True

                if flag != True: # allow correctly formatted triples

                  parsed_objects.append(triple)

                elif strict != True: # penalize each wrong triple under the flexible paradigm

                  penalize += 0.1



        except json.JSONDecodeError:
            pass  # Ignore any invalid JSON objects
  #print(parsed_objects)
  return parsed_objects, penalize

In [15]:
# function to check the validity of a triple against a provided list
# it also allows the flexible paradigm, by changing the ID (last character changed to 1) to check if the model's mistake was only related to wrong number concatenated to the class name
# it makes sure that repetition of triples (the model outputs the same triples more times) are not allowed, while also keeping track of the least penalized triple for a golden one
# it also allows only the verification of the validity of a single ID
# we assume 1 is the highest metric value for a triple, therefore: 0.33 penalty is given for each wrong ID

'''
Parameters
----------
  triple : dict
           The dictionary containing the json object/triple
  only_IDs : bool
             Flag for only checking for the ID correctness, rather of the whole triple
  valid_triples : list
                  The list of valid triples against which to check the triple
  checked_index : list[list,list]
                  A list containing other two lists, one containing the index of the triple in the valid_triples list, while the second stores the associated penalty
  valid_ids : list
              List of valid IDs format, regarding the provided ontology and format
  class_range_rels : list
                     List containing those relationships that require other instance, as suggested by the rdfs:range relationship from the ontology
'''
'''
Return
------
  type : (bool, list[list, list])
         A tuple with the boolean value of the validity evaluation, with the list of checked_index
'''

def check_triple(triple, only_IDs = False, valid_triples = [], checked_index = [[],[]], valid_ids = ['Project1', 'Employee1', 'Status1'], class_range_rels = ['hasStatus', 'hasManager']):

        penalty = 0

        if triple['subject'] in valid_ids:

          if only_IDs:
            return True, checked_index # if only the validity of the ID is asked for

        else:
          triple['subject'] = triple['subject'][:len(triple['subject']) - 1] + '1' # change the last char of the ID to '1', to check for small format errors

          if triple['subject'] in valid_ids:
            penalty += 0.33 # penalize the wrong ID

            if only_IDs:
              return True, checked_index

          elif only_IDs:
            return False, checked_index # if the ID is not valid and we only check its validity, return

        if triple in valid_triples:

          index = valid_triples.index(triple) # store the referred golden triple's index

          if index not in checked_index[0]:
            checked_index[0].append(index) # add it to the list
            checked_index[1].append(penalty) # with its related penalty

            return True, checked_index

          elif checked_index[1][checked_index[0].index(index)] < penalty: # if already in the list, check if this triple has a lower penalty value and replace it
            checked_index[0][checked_index[0].index(index)] = index
            checked_index[1][checked_index[0].index(index)] = penalty

          else:

            return False, checked_index # it means the found triple is not better than an already existing variant for the golden one

        elif triple['relationship'] in class_range_rels: # if not yet valid, check the object's ID, if the relationship requires such object

          triple['object'] = triple['object'][:len(triple['object']) - 1] + '1' # same ID modification as for the subject

          if triple in valid_triples:
            penalty += 0.33 # penalize the wrong object ID format

            # same as above
            index = valid_triples.index(triple)
            if index not in checked_index[0]:
              checked_index[0].append(index)
              checked_index[1].append(penalty)
              return True, checked_index

            elif checked_index[1][checked_index[0].index(index)] < penalty:
              checked_index[0][checked_index[0].index(index)] = index
              checked_index[1][checked_index[0].index(index)] = penalty

            else:

              return False, checked_index

          else:
            return False, checked_index # no valid triple, even after modifications

        else: # no valid triple, even after modifications
          return False, checked_index

In [16]:
# function to calculate the metrics (accuracy, precision, recall) and existence of error or incorrect format triples
# allows both types of paradigms, strict and flexible
# we assume 1 as the maximum metric value for a triple, therefore: we deduct the penalty out of 1 for each triple, where it exists; for the fp_ok triples, we deduct the total number of triples, to allow a small bonus for the model

'''
Parameters
----------
  predicted_triples : list
                      The list of predicted triples
  golden_triples : list
                      The list of golden triples
  alternative_triples : list
                      The list of alternative triples
  fp_ok_triples : list
                      The list of fp_ok triples
  strict : bool
           True means strict measurement of the triples vailidity, False means flexible measurement
  class_range_rels : list
                     List containing those relationships that require other instance, as suggested by the rdfs:range relationship from the ontology
'''
'''
Return
------
  type : (float, float, float, int, int)
         A tuple of accuracy, precision, recall, error, incorrect_format
'''
def calculate_acc_precision_recall(predicted_triples, golden_triples, alternative_triples, fp_ok__triples, strict = True, class_range_rels = ['hasStatus', 'hasManager']):

    correct_count = 0 # number of correct triples
    precision_pass = 0 # number of false positive triples that are allowed under flexible paradigm
    checked_index_strict = {'golden_triples' : [], 'alternative_triples' : [], 'fp_ok__triples' : []} #indexes of verified triples, to avoid duplicate triples allowance
    checked_index_loose = {'golden_triples' : [[],[]], 'alternative_triples' : [[],[]], 'fp_ok__triples' : [[],[]]} # indexes of verified golden triples, along with their associated penalty value, to redeem only the highest scoring triples and avoiding triples' duplicates

    if predicted_triples == 'error':
      return 0, 0, 0, 1, 0 # increase only the error counter by one
    if predicted_triples == 'incorrect format':
      return 0, 0, 0, 0, 1 # increase only the incorrect format counter by one

    for i, triple1 in enumerate(predicted_triples):

      if triple1 in golden_triples:
        index = golden_triples.index(triple1)
        if index not in checked_index_strict['golden_triples']:
          checked_index_strict['golden_triples'].append(index)
          correct_count += 1



      elif strict == False and golden_triples[0] != "None" and triple1 != "None": # enable felxible metrics check

        status, checked_index_loose['golden_triples'] = check_triple(triple1.copy(), valid_triples = golden_triples, checked_index = checked_index_loose['golden_triples'])

        if status == False and alternative_triples: # check if the triple is in the alternative triples

          # a bug (unknown at the moment, does not let the function run properly, unless we print the results of the function first, then use it)
          print(check_triple(triple1.copy(), valid_triples = alternative_triples, checked_index = checked_index_loose['alternative_triples']))
          status, checked_index_loose['alternative_triples'] = check_triple(triple1.copy(), valid_triples = alternative_triples, checked_index = checked_index_loose['alternative_triples'])

          if status == False and fp_ok__triples: # check if the triple is in the fp_ok triples

            valid_ids = ['Employee1', fp_ok__triples[0]['subject']] if fp_ok__triples[0] != 'None' else ['Employee1'] # only allow Employee1 and the provided employee's ID as valid
            status, checked_index_loose['fp_ok__triples'] = check_triple(triple1.copy(), valid_triples = fp_ok__triples, valid_ids = valid_ids, checked_index = checked_index_loose['fp_ok__triples'])

    for k, v in checked_index_loose.items():

      for i, p in zip(v[0], v[1]):

        if i not in checked_index_strict[k]:

          if k in ['golden_triples', 'alternative_triples']:
            correct_count += 1 - p # add as correct only the part of the triple without the penalty

          elif k == 'fp_ok__triples':
            precision_pass += 1 - p # pass only the part of the triple that is valid, without the penalty


    accuracy = correct_count / len(golden_triples) # accuracy is the total number of correct triples over the total number of golden triples
    recall = accuracy # as we measure here the metrics, recall is always equal with accuracy, as we have no other labels apart from being correct
    try: # avoid divison by zero
      precision = correct_count / len(predicted_triples) if strict else correct_count / (len(predicted_triples) - precision_pass) # allow the flexible paradigm, by not counting the fp_ok triples
    except:
      precision = 0

    return accuracy,precision, recall, 0, 0

In [17]:
# function to place the metrics in the relevant list

'''
Parameters
----------
  metrics : list
            The list of metrics
  accuracy : float
             The accuracy metric score
  precision : float
              The precision metric score
  recall : float
           The recall metric score
  penalize : float
             The penalty value
  error : int
          The total number of errors generated by the server
  incorrect_format : int
                     The total number of incorrect formatted model outputs
'''
'''
Return
------
  type : list
         The metrics' list, with the aformentioned metrics placed in it
'''
def place_metrics(metrics, accuracy, precision, recall, penalize, error, incorrect_format):

  metrics[0] += 1 # increase the number of encountered instances
  metrics[1] += accuracy
  metrics[2] += precision
  metrics[3] += recall
  metrics[4] += penalize
  metrics[5] += error
  metrics[6] += incorrect_format

  return metrics

In [18]:
# function to calculate target metrics, such as accuracy, precision, recall, f_penalize, f1, error, incorrect_format

'''
Parameters
----------
  metrics : list
            The list of metrics
'''
'''
Return
------
  type : (float, float, float, float, float, int, int)
  The tuple containing the values for accuracy, precision, recall, f_penalize, f1, error, incorrect_format
'''
def calculate_metrics(metrics):

  nr_instances = metrics[0] - metrics[5] # when errors were generated, those cannot be counted, therefore deduct them from the total number

  if nr_instances:
    accuracy = round(metrics[1]/nr_instances, 2)
    precision = round(metrics[2]/nr_instances, 2)
    recall = round(metrics[3]/nr_instances, 2)
    f_penalize = round(metrics[4]/nr_instances ,2)
    error = metrics[5]
    incorrect_format = metrics[6]
    try:
      f1 = round(2 * (precision * recall) / (precision + recall),2)
    except:
      f1 = 0

  else: # if all instances generated errors
    return 0,0,0,0,0, metrics[5], metrics[6]

  return accuracy, precision, recall, f_penalize, f1, error, incorrect_format

In [19]:
# function to generate the metrics (accuracy, precision, recall, error absolute rate, incorrect format absolute rate) at each combination of levels
# as input, the user has to specify what combination of levels they want their metrics to be calculated at, from:
# 0 - at model level, 1 - at prompt's level, 2 - at prompt's version, 3 - at the class type (Project, Employee, Status), 4 - at input text's type
# eg. providing [0], will have the metrics for each model's performance, [1] gives the performance of all models at a specific prompt level, while [0, 1] will give the permformance of each model at a specific prompt's level

'''
Parameters
----------
  predictions : list
                The list containing the Prompt objects with the predicted outputs
  file : str
         Path of the file where to write the metrics
  levels : list
           The list of coresponding combined levels at which each metrics to be generated; 'all' generates for the combination of all levels
  template_strict : bool
                    True means strict paradigm at find_json_objects(), False means flexible paradigm
  triple_strict : bool
                    True means strict paradigm at calculate_acc_precision_recall(), False means flexible paradigm
  download : bool
             Option to download the text file with the generated metrics
'''

def view_metrics(predictions, file, levels = ['all'], template_strict = False, triple_strict = False, download = True):

  model_acc = {} # store the results at each desired level
  levels_names = []

  for prompts in predictions:

    for kk, vv in prompts.items():

      for i in tqdm(range(len(vv))):

        pred = vv[i]

        for j,p in enumerate(pred.metadata): # check the Prompt's metadata, where the predictions of each model are stored

          model = p['model']
          levels_names = [model, pred.level, pred.version, pred.class_type, pred.text_type]

          metrics_category = ''

          if levels[0] != 'all':

            for I in levels:
              metrics_category += f'_{levels_names[I]}' if I <= len(levels_names) - 1 else ''
          else:
            for l in levels_names:
              metrics_category += f'_{l}'

          metrics_category = metrics_category[1:]

          if metrics_category not in model_acc.keys():
            model_acc[metrics_category] = [0,0,0,0,0,0,0] # nr_instance, acc, precision, recall, penalize, error, incorrect format


          json_objects, penalize = find_json_objects(p['generated_text'], template_strict)

          golden_labels = find_json_objects(pred.golden_labels)

          alternative_labels = find_json_objects(pred.alternative_labels) if find_json_objects(pred.alternative_labels)[0] != "None" else []

          fp_ok_labels = find_json_objects(pred.fp_ok_labels) if find_json_objects(pred.fp_ok_labels)[0] != "None" else []

          accuracy, precision, recall, error, incorrect_format = calculate_acc_precision_recall(json_objects,golden_labels[0], alternative_labels[0], fp_ok_labels[0], triple_strict)
          model_acc[metrics_category] = place_metrics(model_acc[metrics_category], accuracy, precision, recall, penalize, error, incorrect_format)

  f = open(file, 'a')
  header = f'\n---template strict {template_strict}, triple strict {triple_strict}---'
  f.write(header)
  f.write('\n\n')

  print(header)


  for k,v in model_acc.items():

    print(v)
    nr_instances = v[0]
    accuracy, precision, recall, f_penalize, f1, error, incorrect_format = calculate_metrics(v)

    model_acc[k] = [nr_instances, accuracy, precision, recall, f_penalize, error, incorrect_format]

    results = f'Model: {k} \n accuracy: {accuracy};\n precision: {precision};\n recall: {recall};\n f1 score: {f1};\n penalize: {f_penalize}; \n nr of instances: {nr_instances}; \n incorrect output format: {incorrect_format}; \n error: {error}\n'
    print(results)

    f.write(results)
    f.write('\n')

  f.close()
  if download:
    files.download(file)

In [20]:
# function to call each model from the checkpoint list via API, store their results and metadata, and generate the list of predicted Prompt objects
# the input checkpoint list must be [{'model' : 'exact api model name', 'max_context_window' : int (optional)}, ... ]

'''
Parameters
----------
  checkpoint : list
               A list containing dictionaries of models to be queried
  all_prompts : list
               The list containing all the Prompts objects
  file : str
         Path to the file where to write the serialized Prompt objects
  output_template : str
                    A string that is added to the final prompt for marking the beginning of the output from the model
  download : bool
             Option to download the text file with the serialized Prompt objects
'''
'''
Return
------
  type : list
         The list of all generated prompts
'''

def query(checkpoint, all_prompts, file, output_template = " Output:", download = True):

  predictions = []
  gpt_counter = 0 # to count how many calls were made to gpt models

  try:
    for d in all_prompts:

      for model, prompts in d.items():

        prompts_per_model = {}
        prompts_per_model[model] = []

        for prompt in tqdm(prompts):

          messages = prompt.messages

          messages[1]['content'] = messages[1]['content'] + output_template # output format to be always placed at the second role message content

          for cp in checkpoint: # for each model

            model_name = cp['model']

            if model == model_name or model == 'all':

              if model_name.startswith('gpt'):

                client = OpenAI(api_key = userdata.get('GPT_TOKEN')) # load YOUR OWN KEY; here stored as a secret in the google colab

                try:
                  completion = client.chat.completions.with_raw_response.create(
                    model=model_name,
                    messages=messages
                  )
                except:
                  metadata['generated_text'] = f"er1ror:" # if errors are generated from the server

                req_headers = ['openai-model', 'openai-processing-ms', 'x-ratelimit-remaining-requests', 'x-ratelimit-remaining-tokens', 'x-ratelimit-reset-requests']

                metadata = {k.replace('openai-',''): v for k, v in completion.headers.items() if k in req_headers}
                metadata['date'] = str(datetime.now()) + ' GMT'
                metadata['model'] = cp['model'] # seems like OpenAI removed the model parameter, therefore we take it from the input list

                generated_response = completion.parse()
                metadata['system-fingerprint'] = generated_response.system_fingerprint
                metadata['prompt'] = messages[0]['content'] + messages[1]['content']

                gpt_response = generated_response.choices[0].message.content # get the object that `chat.completions.create()` would have returned

                metadata['generated_text'] = gpt_response
                metadata['tokens'] = num_tokens_from_messages_openai(messages)

                prompt.append_metadata(metadata)

                gpt_counter += 1

                if metadata['x-ratelimit-remaining-requests'] == '1': # if we reach the tire's limit, prompt and stop the call
                  print('gpt counter: ', gpt_counter)
                  print('remaining tokens: ', metadata['x-ratelimit-remaining-tokens'])
                  return 'GPT exhausted'

              else: # it's not an OpenAI model, but some model from HG

                tokenizer = AutoTokenizer.from_pretrained(cp["model"])

                try:
                  tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

                except TemplateError:
                  final_messages = [{'role' : 'user', 'content' : ''}]
                  for m in messages:
                    final_messages[0]['content'] += f" {m['content']}"
                  tokenized_chat = tokenizer.apply_chat_template(final_messages, tokenize=False, add_generation_prompt=True)

                API_URL = f"https://api-inference.huggingface.co/models/{cp['model']}"
                # load YOUR OWN TOKEN from HG
                headers = {"Authorization": f"Bearer {userdata.get('HF_TOKEN')}", "x-compute-type" : "cpu+optimized"} # to make sure we get the optimized compute option, if available

                payload = {'inputs' : tokenized_chat, 'parameters' : {'return_full_text' : False, 'max_new_tokens' : None}, 'options' : {'use_cache' : False}}

                metadata = {'date' : str(datetime.now()) + ' GMT'}
                metadata['prompt'] = tokenized_chat
                metadata['model'] = cp['model']

                response = requests.post(API_URL, headers=headers, json=payload)

                metadata['processing-ms'] = round(response.elapsed.microseconds / 1000)

                try:
                  metadata['generated_text'] = response.json()[0]['generated_text']
                except KeyError:
                  metadata['generated_text'] = f"er1ror: {response.json()['error']}" # to avoid any generated text that starts with error from being counted as wrong

                metadata['tokens'] = num_tokens_from_messages(tokenized_chat, tokenizer)

                prompt.append_metadata(metadata)

          prompts_per_model[model].append(prompt)
          write_prompt(prompt, file)

        predictions.append(prompts_per_model)
  except: # in case any unknown error occurs, to be able to download and return the predictions until that point
    print('Error')

    if download:
      files.download(file)

    return predictions

  if download:
    files.download(file)

  return predictions

In [21]:
# same purpose as the above query function, but HF models need to be locally loaded and referenced here, in the function. OpenAI models are still prompted through API calls
# the input checkpoint list must be [{'model' : 'exact api model name', 'max_context_window' : int (optional)}, ... ]

'''
Parameters
----------
  checkpoint : list
               A list containing dictionaries of models to be queried
  all_prompts : list
               The list containing all the Prompts objects
  file : str
         Path to the file where to write the serialized Prompt objects
  output_template : str
                    A string that is added to the final prompt for marking the beginning of the output from the model
  download : bool
             Option to download the text file with the serialized Prompt objects
'''
'''
Return
------
  type : list
         The list of all generated prompts
'''

def query2(checkpoint, all_prompts, file, tmodel,output_template = " Output:", download = True):

  predictions = []
  gpt_counter = 0 # to count how many calls were made to gpt models

  try:
    for d in all_prompts:

      for model, prompts in d.items():

        prompts_per_model = {}
        prompts_per_model[model] = []

        for prompt in tqdm(prompts):

          messages = prompt.messages

          messages[1]['content'] = messages[1]['content'] + output_template # output format to be always placed at the second role message content

          for cp in checkpoint: # for each model

            model_name = cp['model']

            if model == model_name or model == 'all':

              if model_name.startswith('gpt'):

                client = OpenAI(api_key = userdata.get('GPT_TOKEN')) # load YOUR OWN KEY; here stored as a secret in the google colab

                try:
                  completion = client.chat.completions.with_raw_response.create(
                    model=model_name,
                    messages=messages
                  )
                except:
                  metadata['generated_text'] = f"er1ror:" # if errors are generated from the server

                req_headers = ['openai-model', 'openai-processing-ms', 'x-ratelimit-remaining-requests', 'x-ratelimit-remaining-tokens', 'x-ratelimit-reset-requests']

                metadata = {k.replace('openai-',''): v for k, v in completion.headers.items() if k in req_headers}
                metadata['date'] = str(datetime.now()) + ' GMT'
                metadata['model'] = cp['model'] # seems like OpenAI removed the model parameter, therefore we take it from the input list

                generated_response = completion.parse()
                metadata['system-fingerprint'] = generated_response.system_fingerprint
                metadata['prompt'] = messages[0]['content'] + messages[1]['content']

                gpt_response = generated_response.choices[0].message.content # get the object that `chat.completions.create()` would have returned

                metadata['generated_text'] = gpt_response
                metadata['tokens'] = num_tokens_from_messages_openai(messages)

                prompt.append_metadata(metadata)

                gpt_counter += 1

                if metadata['x-ratelimit-remaining-requests'] == '1': # if we reach the tire's limit, prompt and stop the call
                  print('gpt counter: ', gpt_counter)
                  print('remaining tokens: ', metadata['x-ratelimit-remaining-tokens'])
                  return 'GPT exhausted'

              else: # it's not an OpenAI model, but some model from HG

                tokenizer = AutoTokenizer.from_pretrained(cp["model"])

                try:
                  tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

                except TemplateError:
                  final_messages = [{'role' : 'user', 'content' : ''}]
                  for m in messages:
                    final_messages[0]['content'] += f" {m['content']}"
                  tokenized_chat = tokenizer.apply_chat_template(final_messages, tokenize=False, add_generation_prompt=True)


                # load the model in a 4bit configuration to use QLora; this is how we trained the models, but you can also load them in 8bit or full if enough resources are available


                #tokenizer.pad_token_id = tokenizer.eos_token_id
                pipe = pipeline(task="text-generation",
                        model=tmodel,
                        tokenizer=tokenizer,
                        max_new_tokens=400,
                        return_full_text=False,
                        batch_size = 1
                        )

                #inputs = tokenizer(tokenized_chat, return_tensors="pt").to(tmodel.device)


                #beam_outputs = tmodel.generate(
                #                              **inputs,
                #                              max_new_tokens=400,
                #                              num_beams=1,
                #                              num_return_sequences=1,
                #                              early_stopping=True,
                #                          )


                #predictions = []
                #for i, beam_output in enumerate(beam_outputs):
                #  beam_pred = tokenizer.decode(beam_output[len(inputs['input_ids'][0]):], skip_special_tokens=True)


                #API_URL = f"https://api-inference.huggingface.co/models/{cp['model']}"
                # load YOUR OWN TOKEN from HG
                #headers = {"Authorization": f"Bearer {userdata.get('HF_TOKEN')}", "x-compute-type" : "cpu+optimized"} # to make sure we get the optimized compute option, if available

                #payload = {'inputs' : tokenized_chat, 'parameters' : {'return_full_text' : False, 'max_new_tokens' : None}, 'options' : {'use_cache' : False}}

                metadata = {'date' : str(datetime.now()) + ' GMT'}
                metadata['prompt'] = tokenized_chat
                metadata['model'] = cp['model']
                try:
                  response = pipe(tokenized_chat)
                  metadata['generated_text'] = response[0]['generated_text']
                except:
                    metadata['generated_text'] = f"er1ror: {response}"
                #metadata['generated_text'] = predictions
                #response = requests.post(API_URL, headers=headers, json=payload)

                #metadata['processing-ms'] = round(response.elapsed.microseconds / 1000)

                #try:
                #  metadata['generated_text'] = response.json()[0]['generated_text']
                #except KeyError:
                #  metadata['generated_text'] = f"er1ror: {response.json()['error']}" # to avoid any generated text that starts with error from being counted as wrong

                metadata['tokens'] = num_tokens_from_messages(tokenized_chat, tokenizer)

                prompt.append_metadata(metadata)

          prompts_per_model[model].append(prompt)
          write_prompt(prompt, file)

        predictions.append(prompts_per_model)
  except BaseException as e: # in case any unknown error occurs, to be able to download and return the predictions until that point
    print(e)

    if download:
      files.download(file)

    return predictions

  if download:
    files.download(file)

  return predictions

## Generate dataset (enter to set variables)

In [ ]:
# set your own preferences

download_files = False # if you want to download the serialized dataset
file_to_generate_from = 'templates_easy.txt' # file path to the templates

In [ ]:
dataset = []
dataset = generate_dataset(file_to_generate_from, dataset = dataset, download = download_files)

## Load dataset (enter to set variables)

In [64]:
# set your own preferences

file_to_load_from = 'templates_easy_final.txt' # file path to the serialized dataset

In [ ]:
dataset = from_text_to_dataset(file_to_load_from)

## Generate prompts (enter to set variables)

In [66]:
# RUN THIS CELL FOR TODSET

# provide the ontology
ontology = '''"
    @prefix : <http://john.com/project> .
    @prefix owl: <http://www.w3.org/2002/07/owl#> .
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

    <http://john.com/project> rdf:type owl:Ontology .

    :hasClass rdf:type owl:ObjectProperty ;
        rdfs:domain :Project ;
        rdfs:range xsd:string .

    :hasCode rdf:type owl:ObjectProperty ;
        rdfs:domain :Project ;
        rdfs:range xsd:string .

    :hasManager rdf:type owl:ObjectProperty ;
        rdfs:domain :Project ;
        rdfs:range  :Employee .

    :hasName rdf:type owl:ObjectProperty ;
        rdfs:domain owl:Thing ;
        rdfs:range xsd:string .

    :hasRole rdf:type owl:ObjectProperty ;
        rdfs:domain :Employee ;
        rdfs:range xsd:string .

    :hasStatus rdf:type owl:ObjectProperty ;
        rdfs:domain :Project ;
        rdfs:range :Status .


    :Employee rdf:type owl:Class ;
        rdfs:subClassOf owl:Thing.

    :Status rdf:type owl:Class;
        rdfs:subClassOf owl:Thing.

    :Project rdf:type owl:Class;
        rdfs:subClassOf owl:Thing."'''

# provide examples for each level;

# here for level 4.1, for ICL style
example_labels = f'Example: \n Input text: Insert a project that has code 0A0, name it ColourRun, its class is Python and manager is Maia. \n Output: [{{"subject" : "Project1", "relationship" : "rdf:type" , "object" : "Project" }}, {{"subject" : "Project1", "relationship" : "hasCode" , "object" : "0A0" }}, {{"subject" : "Project1", "relationship" : "hasClass" , "object" : "Python" }}, {{"subject" : "Project1", "relationship" : "hasManager", "object" : "Employee1" }}, {{"subject" : "Employee1", "relationship" : "hasName", "object" : "Maia" }}, {{"subject" : "Employee1", "relationship" : "rdf:type", "object" : "Employee" }}, {{"subject" : "Project1", "relationship" : "hasName" , "object" : "ColourRun" }}] \n'

# here for level 4.2, for COT style
example_labels_cot = f'Example: \n Input text: Insert a project that has code 0A0, name it ColourRun, its class is Python and manager is Maia. \n 1. In the ontology, three classes are found: Project, Employee, and Status. In the input text, the word "project" may refer to the class Project. \n 2. The ID of the instance is the name of the class concatenated with "1", therefore the final ID is Project1. \n 3. The type relationship creates the triple :Project1 rdf:type :Project. \n 4. Given triples where the rdfs:domain is :Project or :Thing, specific relationships are hasName, hasCode, hasClass, hasManager, and hasStatus. Within the input text, relationship keywords like "name", "class", "code", and "manager" may lead to the following relationships: hasName, hasCode, hasClass, and hasManager. \n 5. The possible object for each relationship may be words that semantically and/or syntactically relate to the relationship keywords. Therefore, we can extract: for hasName the word "ColourRun", for hasCode the word "0A0", for hasClass the word "Python", and for hasManager the word "Maia". \n 6. For hasManager, the ontology specifies an Employee instance, therefore a new instance of type Employee with ID Employee1 is created, and the object "Maia" may be the name of it. Each triple has to be put in a JSON object respecting the provided template. All JSON objects must be put in a list. \n Output: [{{"subject" : "Project1", "relationship" : "rdf:type" , "object" : "Project" }}, {{"subject" : "Project1", "relationship" : "hasCode" , "object" : "0A0" }}, {{"subject" : "Project1", "relationship" : "hasClass" , "object" : "Python" }}, {{"subject" : "Project1", "relationship" : "hasManager", "object" : "Employee1" }}, {{"subject" : "Employee1", "relationship" : "hasName", "object" : "Maia" }}, {{"subject" : "Employee1", "relationship" : "rdf:type", "object" : "Employee" }}, {{"subject" : "Project1", "relationship" : "hasName" , "object" : "ColourRun" }}] \n'

# here for level 4.1, for ICL style of no golden triples
example_none = f'Example: \n Input text: Insert a program with code as A43, manager as Michael and class as Java. \n Output: "None" \n'

# here for level 4.2, for COT style of no golden triples
example_none_cot = f'Example: \n Input text: Insert a program with code as A43, manager as Michael and class as Java. \n 1. In the ontology, three classes are found: Project, Employee, and Status. In the input text, there are no words to suggest these classes, therefore the output is "None". \n Output: "None" \n'

In [32]:
# RUN THIS CELL FOR FOOTBALLSET

# provide the ontology
ontology = '''"
  @prefix dc: <http://purl.org/dc/elements/1.1/> .
  @prefix onto: <https://cenguix.github.io/Text2KGBench/ont_15_sportsteam#> .
  @prefix owl: <http://www.w3.org/2002/07/owl#> .
  @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
  @prefix ontorels: <https://cenguix.github.io/Text2KGBench/ont_15_sportsteam/relations#> .
  @prefix ontoconcepts: <https://cenguix.github.io/Text2KGBench/ont_15_sportsteam/concepts#> .
  @prefix ontoliterals: <https://cenguix.github.io/Text2KGBench/ont_15_sportsteam/literals#> .

  <http://example.com/ontology> a owl:Ontology ;
      rdfs:label "SportsTeam Ontology" ;
      dc:creator <http://orcid.org/0000-0001-7197-0766>,
          <http://orcid.org/0000-0003-4303-983X>,
          <http://orcid.org/0000−0003−1707−4842> ;
      rdfs:comment "An ontology used in the Text2KGBench, modified for our work." ;
      owl:versionIRI onto:v1 .

  ontorels:birthPlace a owl:ObjectProperty ;
      rdfs:label "birthPlace" ;
      rdfs:domain ontoconcepts:Person ;
      rdfs:range ontoconcepts:Place .

  ontorels:capital a owl:ObjectProperty ;
      rdfs:label "capital" ;
      rdfs:domain ontoconcepts:Country ;
      rdfs:range ontoconcepts:City .

  ontorels:chairmanTitle a owl:ObjectProperty ;
      rdfs:label "chairmanTitle" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoliterals:string .

  ontorels:champions a owl:ObjectProperty ;
      rdfs:label "champions" ;
      rdfs:domain ontoconcepts:League ;
      rdfs:range ontoconcepts:SportsTeam .

  ontorels:city a owl:ObjectProperty ;
      rdfs:label "city" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:City .

  ontorels:club a owl:ObjectProperty ;
      rdfs:label "club" ;
      rdfs:domain ontoconcepts:Athlete ;
      rdfs:range ontoconcepts:SportsTeam .

  ontorels:country a owl:ObjectProperty ;
      rdfs:label "country" ;
      rdfs:domain ontoconcepts:City ;
      rdfs:range ontoconcepts:Country .

  ontorels:demonym a owl:ObjectProperty ;
      rdfs:label "demonym" ;
      rdfs:domain ontoconcepts:Country ;
      rdfs:range ontoconcepts:Demonym .

  ontorels:fullName a owl:ObjectProperty ;
      rdfs:label "fullName" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoliterals:string .

  ontorels:ground a owl:ObjectProperty ;
      rdfs:label "ground" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:Ground .

  ontorels:hasName a owl:ObjectProperty ;
      rdfs:label "hasName" ;
      rdfs:domain owl:Thing ;
      rdfs:range ontoliterals:string .

  ontorels:isPartOf a owl:ObjectProperty ;
      rdfs:label "isPartOf" ;
      rdfs:domain ontoconcepts:City ;
      rdfs:range ontoconcepts:Country .

  ontorels:language a owl:ObjectProperty ;
      rdfs:label "language" ;
      rdfs:domain ontoconcepts:Country ;
      rdfs:range ontoconcepts:Language .

  ontorels:leader a owl:ObjectProperty ;
      rdfs:label "leader" ;
      rdfs:domain ontoconcepts:Country ;
      rdfs:range ontoconcepts:Person .

  ontorels:league a owl:ObjectProperty ;
      rdfs:label "league" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:League .

  ontorels:location a owl:ObjectProperty ;
      rdfs:label "location" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:Place .

  ontorels:manager a owl:ObjectProperty ;
      rdfs:label "manager" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:Person .

  ontorels:mayor a owl:ObjectProperty ;
      rdfs:label "mayor" ;
      rdfs:domain ontoconcepts:City ;
      rdfs:range ontoconcepts:Person .

  ontorels:nickname a owl:ObjectProperty ;
      rdfs:label "nickname" ;
      rdfs:domain ontoconcepts:Person ;
      rdfs:range ontoliterals:string .

  ontorels:numberOfMembers a owl:ObjectProperty ;
      rdfs:label "numberOfMembers" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoliterals:number .

  ontorels:operator a owl:ObjectProperty ;
      rdfs:label "operator" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:Company .

  ontorels:owner a owl:ObjectProperty ;
      rdfs:label "owner" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:Person .

  ontorels:season a owl:ObjectProperty ;
      rdfs:label "season" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:Season .

  ontorels:state a owl:ObjectProperty ;
      rdfs:label "state" ;
      rdfs:domain ontoconcepts:City ;
      rdfs:range ontoconcepts:State .

  ontorels:tenant a owl:ObjectProperty ;
      rdfs:label "tenant" ;
      rdfs:domain ontoconcepts:SportsTeam ;
      rdfs:range ontoconcepts:Tenant .

  ontoconcepts:Company a owl:Class ;
      rdfs:label "Company" .

  ontoconcepts:Demonym a owl:Class ;
      rdfs:label "Demonym" .

  ontoconcepts:Ground a owl:Class ;
      rdfs:subClassOf ontoconcepts:Country ;
      rdfs:subClassOf ontoconcepts:City ;
      rdfs:label "Ground" .

  ontoconcepts:Language a owl:Class ;
      rdfs:label "Language" .

  ontoconcepts:Season a owl:Class ;
      rdfs:label "Season" .

  ontoconcepts:State a owl:Class ;
      rdfs:label "State" .

  ontoconcepts:Tenant a owl:Class ;
      rdfs:label "Tenant" .

  ontoconcepts:Athlete a owl:Class ;
      rdfs:subClassOf ontoconcepts:Person ;
      rdfs:label "Athlete" .

  ontoconcepts:League a owl:Class ;
      rdfs:label "League" .

  ontoconcepts:Place a owl:Class ;
      rdfs:label "Place" .

  ontoconcepts:City a owl:Class ;
      rdfs:subClassOf ontoconcepts:Place ;
      rdfs:label "City" .

  ontoconcepts:Country a owl:Class ;
      rdfs:label "Country" ;
      rdfs:subClassOf ontoconcepts:Place .

  ontoconcepts:Person a owl:Class ;
      rdfs:label "Person" .

  ontoconcepts:SportsTeam a owl:Class ;
      rdfs:label "SportsTeam" ."'''

# provide examples for each level;

# here for level 4.1, for ICL style
example_labels = f'Example: \n Input text: Athlete Marco Reus was born in Germany, and currently plays for LA Galaxy. He is nicknamed Marcinho. \n Output: [{{"subject": "Country1", "relationship": "rdf:type", "object": "Country"}}, {{"subject": "Country1", "relationship": "hasName", "object": "Germany"}}, {{"subject": "Person1", "relationship": "rdf:type", "object": "Person"}}, {{"subject": "Person1", "relationship": "hasName", "object": "Marco Reus"}}, {{"subject": "Person1", "relationship": "birthPlace", "object": "Country1"}}, {{"subject": "SportsTeam1", "relationship": "rdf:type", "object": "SportsTeam"}}, {{"subject": "SportsTeam1", "relationship": "hasName", "object": "LA Galaxy"}}, {{"subject": "Person1", "relationship": "club", "object": "SportsTeam1"}}, {{"subject": "Person1", "relationship": "nickname", "object": "Marcinho"}}] \n'

# here for level 4.2, for COT style
example_labels_cot = f'Example: \n Input text: Athlete Marco Reus was born in Germany, and currently plays for LA Galaxy. He is nicknamed Marcinho. \n  1. In the ontology, 14 classes are found: "SportsTeam", "Athlete", "Country", "City", "Person", "Place", "Demonym", "League", "Tenant", "Language", "Season", "State", "Ground", "Company". In the input text, the word "Athlete" may refer to the class Athlete. This class is also a subclass of Person. Depending on the detected relationships domains, we have to decide whether Person or Athlete is the right choice. \n 2. The ID of the instance is the name of the class concatenated with "1", therefore the final ID can be either Athlete1 or Person1. \n 3. The type relationship creates either the triple :Athlete1 rdf:type :Athlete or :Person1 rdf:type :Person. \n 4. Given triples where the rdfs:domain is :Athlete, :Thing or :Person (because the Athlete class is a subclass of Person), specific relationships are club, hasName, nickname and birthPlace. Within the input text, there are keywords like "born in", "plays for", and "nicknamed" that may lead to the following relationships: club (because playing for hints that the player is part of a football club), nickname and birthPlace. Also, we have his name which leads to the hasName relationship. \n Revision to step 2. We can now say that the instance is of type Person, because the domains of most relationships it is part of are requesting Person instances. \n 5. The possible object for each relationship may be words that semantically and/or syntactically relate to the relationship keywords. Therefore, we can extract: for hasName the words "Marco Reus", for club the words "LA Galaxy", for nickname the word "Marcinho", and for birthPlace the word "Germany". \n 6. For club, the ontology specifies a SportsTeam instance, therefore a new instance of type SportsTeam with ID SportsTeam1 is created, and the object "LA Galaxy" may be the name of it. For birthPlace, the ontology specifies a Place instance. We can see that the detected object is "Germany" which is known to be a country. In the ontology, we can see that Country is a subclass of Place, therefore we can use Country as its class. Depending on the other related relationships, Place might have been the better choice, similar to how we have selected Person over Athlete. A new instance of type Country with ID Country1 is created, and the object "Germany" may be the name of it. Each triple has to be put in a JSON object respecting the provided template. All JSON objects must be put in a list. \n Output: [{{"subject": "Country1", "relationship": "rdf:type", "object": "Country"}}, {{"subject": "Country1", "relationship": "hasName", "object": "Germany"}}, {{"subject": "Person1", "relationship": "rdf:type", "object": "Person"}}, {{"subject": "Person1", "relationship": "hasName", "object": "Marco Reus"}}, {{"subject": "Person1", "relationship": "birthPlace", "object": "Country1"}}, {{"subject": "SportsTeam1", "relationship": "rdf:type", "object": "SportsTeam"}}, {{"subject": "SportsTeam1", "relationship": "hasName", "object": "LA Galaxy"}}, {{"subject": "Person1", "relationship": "club", "object": "SportsTeam1"}}, {{"subject": "Person1", "relationship": "nickname", "object": "Marcinho"}}] \n'

# here for level 4.1, for ICL style of no golden triples
example_none = f'Example: \n Input text: The Video Assistant Referee (VAR) technology is very useful in Premier League mathces. \n Output: "None" \n'

# here for level 4.2, for COT style of no golden triples
example_none_cot = f'Example: \n Input text: The Video Assistant Referee (VAR) technology is very useful in Premier League mathces. \n 1. In the ontology, 14 classes are found: "SportsTeam", "Athlete", "Country", "City", "Person", "Place", "Demonym", "League", "Tenant", "Language", "Season", "State", "Ground", "Company". In the input text, there are no words to suggest these classes, therefore the output is "None". \n Output: "None" \n'

In [ ]:
# provide the system prompts, following the same format as below;

system_prompts = {
    "all" : # all type of models
     {
      0.0: # version
         [  # level
          { 1 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. It may or may not contain references to an instance of a class provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instance from the input text. Each instance should be identified by an ID, using the format :Class + "1", where class is the name of the detected class and + is concatenation. Respond only with the triples. The output format for a triple is :Subject :Relationship "Object". If no triple is detected, output "None".\n Provided ontology: {ontology} \n'},
          { 2 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. It may or may not contain references to an instance of a class provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instance from the input text. Each instance should be identified by an ID, using the format :Class + "1", where class is the name of the detected class and + is concatenation. Each extracted triple that is related to the detected instance should use the ID as the subject. The first triple you always create is between the ID and the class type, using the rdf:type relationship. Respond only with the triples. The output format for a triple is :Subject :Relationship "Object". If no triple is detected, output "None". \n Provided ontology: {ontology} \n'},
          { 3 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. It may or may not contain references to an instance of a class provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instance from the input text, as follows: \n 1.Check if a class from the provided ontology is mentioned in the input text. Search in the provided ontology for triples with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. The input text should contain the name of a class from the ontology, even if misspelled. If the class is not known, stop the process and output "None". \n 2. Each instance should be identified by an ID, using the format :Class + "1", where class is the name of the detected class and + is concatenation. Each extracted triple that is related to the detected instance should use the ID as the subject. \n 3. The first triple you always create is between the ID and the class type, using the rdf:type relationship. \n 4. Check whether any relationships were mentioned in the input text, regarding the detected class. For this, search in the ontology for the triples of form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship regarding the detected class. Also, search for the triples of form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes, even if it is not directly mentioned. \n 5. For each detected relationship, extract the possible mentioned object in the input text, regarding the detected instance. \n 6. Respond only with the triples. The output format for a triple is :Subject :Relationship "Object". If no triple is detected, output "None". \n Provided ontology: {ontology} \n'},
          { 4.1 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. It may or may not contain references to an instance of a class provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instance from the input text, as follows: \n 1.Check if a class from the provided ontology is mentioned in the input text. Search in the provided ontology for triples with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. The input text should contain the name of a class from the ontology, even if misspelled. If the class is not known, stop the process and output "None". \n 2. Each instance should be identified by an ID, using the format :Class + "1", where class is the name of the detected class and + is concatenation. Each extracted triple that is related to the detected instance should use the ID as the subject. \n 3. The first triple you always create is between the ID and the class type, using the rdf:type relationship. \n 4. Check whether any relationships were mentioned in the input text, regarding the detected class. For this, search in the ontology for the triples of form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship regarding the detected class. Also, search for the triples of form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes, even if it is not directly mentioned. \n 5. For each detected relationship, extract the possible mentioned object in the input text, regarding the detected instance. \n 6. Respond only with the triples. The output format for a triple is :Subject :Relationship "Object". If no triple is detected, output "None". \n Provided ontology: {ontology} \n Example: \n Input text: Insert a project that has code 0A0, name it ColourRun, its class is Python and manager is Maia. \n Triples: :Project1 rdf:type :Project; \n :Project1 :hasCode "0A0; \n :Project1 :hasName "ColourRun"; \n :Project1 :hasClass "Python"; \n :Project1 :hasManager :Employee1; \n :Employee1 :hasName "Maia".\n'},
          { 4.2 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. It may or may not contain references to an instance of a class provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instance from the input text, as follows: \n 1.Check if a class from the provided ontology is mentioned in the input text. Search in the provided ontology for triples with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. The input text should contain the name of a class from the ontology, even if misspelled. If the class is not known, stop the process and output "None". \n 2. Each instance should be identified by an ID, using the format :Class + "1", where class is the name of the detected class and + is concatenation. Each extracted triple that is related to the detected instance should use the ID as the subject. \n 3. The first triple you always create is between the ID and the class type, using the rdf:type relationship. \n 4. Check whether any relationships were mentioned in the input text, regarding the detected class. For this, search in the ontology for the triples of form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship regarding the detected class. Also, search for the triples of form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes, even if it is not directly mentioned. \n 5. For each detected relationship, extract the possible mentioned object in the input text, regarding the detected instance. \n 6. Respond only with the triples. The output format for a triple is :Subject :Relationship "Object". If no triple is detected, output "None".  \n Provided ontology: {ontology} \n Example: \n Input text: Insert a project that has code 0A0, name it ColourRun, its class is Python and manager is Maia. \n 1. In the ontology, three class triples are found: Project, Employee, and Status. In the input text, the word "project" may refer to the class Project. \n 2. The ID of the instance is the name of the class concatenated with "1", therefore the final ID is :Project1. \n 3. The type relationship creates the triple :Project1 rdf:type :Project. \n 4. Given triples where the rdfs:domain is :Project or :Thing, specific relationships are hasName, hasCode, hasClass, hasManager, and hasStatus. Within the input text, relationship keywords like "name", "class", "code", and "manager" may lead to the following relationships: hasName, hasCode, hasClass, and hasManager. \n 5. The possible object for each relationship may be words that semantically and/or syntactically relate to the relationship keywords. Therefore, we can extract: for hasName the word "ColourRun", for hasCode the word "0A0", for hasClass the word "Python", and for hasManager the word "Maia". \n 6. For hasManager, the ontology specifies an Employee instance, therefore a new instance of type Employee with ID :Employee1 is created, and the object "Maia" may be the name of it. Triples: :Project1 rdf:type :Project; \n :Project1 :hasCode "0A0; \n :Project1 :hasName "ColourRun"; \n :Project1 :hasClass "Python"; \n :Project1 :hasManager :Employee1; \n :Employee1 :hasName "Maia".\n'},
          ],
      1.0: # version
         [  # level
          { 1 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. The input text may or may not contain references to instances of classes provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instances from the input text. Each instance should be identified by an ID, using the format "Class" + "1", where "Class" is the name of the detected class and + is concatenation. Put each triple in a JSON object, as follows: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond only with the JSON object(s) in a list. If no triple is detected, output "None".\n Provided ontology: {ontology} \n'},
          { 2 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. The input text may or may not contain references to instances of classes provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instances from the input text. Each instance should be identified by an ID, using the format "Class" + "1", where "Class" is the name of the detected class and + is concatenation. A triple you always create is between the ID and the class type, using the rdf:type relationship. Put each triple in a JSON object, as follows: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond only with the JSON object(s) in a list. If no triple is detected, output "None".\n Provided ontology: {ontology} \n'},
          { 3 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. The input text may or may not contain references to instances of classes provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instances from the input text, as follows: \n 1. Check if a class from the provided ontology is mentioned in the input text. Search in the provided ontology for triples with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. The input text should contain the name of a class from the ontology, even if misspelled. If the class is not known, stop the process and output "None". \n 2. Each instance should be identified by an ID, using the format "Class" + "1", where "Class" is the name of the detected class and + is concatenation. Each extracted triple that is related to an instance should use the instance\'s ID as the subject. \n 3. A triple you always create is between the ID and the class type, using the rdf:type relationship. \n 4. Check whether any relationships were mentioned in the input text, regarding the detected class. For this, search in the ontology for the triples of form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship regarding the detected class. Also, search for the triples of form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes, even if it is not directly mentioned. \n 5. For each detected relationship, extract the possible mentioned object in the input text, regarding the detected instance. \n 6. Put each triple in a JSON object, as follows: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond only with the JSON object(s) in a list. If no triple is detected, output "None".\n Provided ontology: {ontology} \n'},
          { 4.1 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. The input text may or may not contain references to instances of classes provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instances from the input text, as follows: \n 1. Check if a class from the provided ontology is mentioned in the input text. Search in the provided ontology for triples with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. The input text should contain the name of a class from the ontology, even if misspelled. If the class is not known, stop the process and output "None". \n 2. Each instance should be identified by an ID, using the format "Class" + "1", where "Class" is the name of the detected class and + is concatenation. Each extracted triple that is related to an instance should use the instance\'s ID as the subject. \n 3. A triple you always create is between the ID and the class type, using the rdf:type relationship. \n 4. Check whether any relationships were mentioned in the input text, regarding the detected class. For this, search in the ontology for the triples of form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship regarding the detected class. Also, search for the triples of form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes, even if it is not directly mentioned. \n 5. For each detected relationship, extract the possible mentioned object in the input text, regarding the detected instance. \n 6. Put each triple in a JSON object, as follows: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond only with the JSON object(s) in a list. If no triple is detected, output "None".\n Provided ontology: {ontology} \n '},
          { 4.2 : f'You are a Knowledge Graph Expert. A domain ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. The input text may or may not contain references to instances of classes provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instances from the input text, as follows: \n 1. Check if a class from the provided ontology is mentioned in the input text. Search in the provided ontology for triples with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. The input text should contain the name of a class from the ontology, even if misspelled. If the class is not known, stop the process and output "None". \n 2. Each instance should be identified by an ID, using the format "Class" + "1", where "Class" is the name of the detected class and + is concatenation. Each extracted triple that is related to an instance should use the instance\'s ID as the subject. \n 3. A triple you always create is between the ID and the class type, using the rdf:type relationship. \n 4. Check whether any relationships were mentioned in the input text, regarding the detected class. For this, search in the ontology for the triples of form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship regarding the detected class. Also, search for the triples of form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes, even if it is not directly mentioned. \n 5. For each detected relationship, extract the possible mentioned object in the input text, regarding the detected instance. \n 6. Put each triple in a JSON object, as follows: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond only with the JSON object(s) in a list. If no triple is detected, output "None". \n Provided ontology: {ontology} \n '},
          ],
      }
    }

# mistralai/Mistral-7B-Instruct-v0.1 has been removed because it did not reprhase anything in three runs
final_model_prompts = {
    'mistralai/Mixtral-8x7B-Instruct-v0.1':
     {1.0:
          [
              {1: f'You are a specialist in Knowledge Graphs and will be given a domain ontology in the Turtle syntax, enclosed by double quotes. The task involves extracting triples about instances mentioned in a provided natural language text. These instances may belong to classes in the given ontology, with possible relationships between them. Each instance should be identified by a unique ID, constructed by concatenating the class name and the number 1 (e.g., "Class1"). Create a JSON object for each triple, following the format: {{"subject" : ID, "relationship" : value, "object" : value}}. If a triple references another instance, include all assumed triples of that instance as well. The output should be a list of JSON objects. If no triples are detected, respond with "None".\n Provided ontology: {ontology} \n'},
              {2: f'You are a specialist in Knowledge Graphs and will be given a domain ontology in the Turtle syntax, enclosed in double quotes. The task requires analyzing a provided natural language text, which may or may not refer to instances of classes in the given ontology along with specific relationships. Your goal is to identify triples related to the mentioned instances from the input text. Each identified instance should be assigned a unique ID, constructed by concatenating the class name and the number 1 (e.g., "Class1"). Create a triple connecting the ID and the class type using the rdf:type relationship. Present each triple in a JSON object, following the format: {{"subject" : ID, "relationship" : value, "object" : value}}, without the whole IRI term. If any triple refers to another instance, include all inferred triples related to that instance as well. Your response should be a list containing the JSON object(s), or "None" if no triples are detected.\n Provided ontology: {ontology} \n'},
              {3: f'You are a Knowledge Graph Expert. You are given a domain ontology in the form of a Turtle syntax string, enclosed by double quotes. Your task is to extract triples from a provided natural language text, which may or may not contain references to instances of classes in the ontology along with specific relationships. Follow this process:\n1. Identify a class from the provided ontology in the input text. Look for triples in the ontology with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. If the class is not found in the text, even if misspelled, output "None".\n2. Assign an ID to each instance in the format "Class" + "1", where "Class" is the identified class name. Use this ID as the subject for any triples related to the instance.\n3. Create a triple between the ID and the class type using the rdf:type relationship.\n4. Find relationships related to the detected class by searching for triples of the form :RelationshipName rdfs:domain :ClassName and :Relationship rdfs:domain owl:Thing in the ontology.\n5. Extract possible objects from the input text for each detected relationship.\n6. Format each triple as a JSON object with the structure {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, include all assumed triples of that instance as well. Output only the list. If no triples are detected, output "None".\n Provided ontology: {ontology} \n'},
              {4.1: f'You are a Knowledge Graph Expert. You are given a domain ontology in the form of a Turtle syntax string, enclosed by double quotes. Your task is to extract triples from a provided natural language text, which may or may not contain references to instances of classes in the ontology along with specific relationships. Follow this process:\n1. Identify a class from the provided ontology in the input text. Look for triples in the ontology with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. If the class is not found in the text, even if misspelled, output "None".\n2. Assign an ID to each instance in the format "Class" + "1", where "Class" is the identified class name. Use this ID as the subject for any triples related to the instance.\n3. Create a triple between the ID and the class type using the rdf:type relationship.\n4. Find relationships related to the detected class by searching for triples of the form :RelationshipName rdfs:domain :ClassName and :Relationship rdfs:domain owl:Thing in the ontology.\n5. Extract possible objects from the input text for each detected relationship.\n6. Format each triple as a JSON object with the structure {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, include all assumed triples of that instance as well. Output only the list. If no triples are detected, output "None".\n Provided ontology: {ontology} \n'},
              {4.2: f'You are a Knowledge Graph Expert. You are given a domain ontology in the form of a Turtle syntax string, enclosed by double quotes. Your task is to extract triples from a provided natural language text, which may or may not contain references to instances of classes in the ontology along with specific relationships. Follow this process:\n1. Identify a class from the provided ontology in the input text. Look for triples in the ontology with the format :ClassName rdf:type owl:Class, where ClassName is the name of the class. If the class is not found in the text, even if misspelled, output "None".\n2. Assign an ID to each instance in the format "Class" + "1", where "Class" is the identified class name. Use this ID as the subject for any triples related to the instance.\n3. Create a triple between the ID and the class type using the rdf:type relationship.\n4. Find relationships related to the detected class by searching for triples of the form :RelationshipName rdfs:domain :ClassName and :Relationship rdfs:domain owl:Thing in the ontology.\n5. Extract possible objects from the input text for each detected relationship.\n6. Format each triple as a JSON object with the structure {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, include all assumed triples of that instance as well. Output only the list. If no triples are detected, output "None".\n Provided ontology: {ontology} \n'},
            ]
        },
  #  'openchat/openchat-3.5-0106':
  #    {1.0:
  #        [
  #            {1: 'You are a Knowledge Graph Expert. An ontology is provided to you, delimited by double quotes. The syntax used to describe the ontology is Turtle. Your input is a natural language text. The input text may or may not contain references to instances of classes provided in the ontology, together with specific relationships. Given the provided ontology, your task is to extract triples about the mentioned instances from the input text. Each instance should be identified by an ID, using the format "Class" + "1", where "Class" is the name of the detected class and + is concatenation. Put each triple in a JSON object, as follows: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond only with the JSON object(s) in a list. If no triple is detected, output "None".\n Provided ontology: {ontology} \n'},
  #            {2: 'You are a Knowledge Graph Expert. You are given a domain ontology, which is delimited by double quotes and described using the Turtle syntax. Your input is a natural language text that may or may not contain references to instances of classes provided in the ontology, along with specific relationships. Your task is to extract triples about the mentioned instances from the input text. Each instance should be identified by an ID, using the format "Class" + "1", where "Class" is the name of the detected class and + is concatenation. You must always create a triple between the ID and the class type, using the rdf:type relationship. \n\nPut each triple in a JSON object, with the format: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, include all triples you assumed of that instance too. Respond only with the JSON object(s) in a list. If no triple is detected, output "None".\nProvided ontology: {ontology} \n'},
  #            {3: f'You are a Knowledge Graph Expert. You are given a domain ontology in Turtle syntax, delimited by double quotes. Your task is to process a natural language text input, extracting triples about instances mentioned in the text based on the provided ontology. The input text may contain references to instances of classes in the ontology, along with specific relationships. Your task is to:\n1. Identify if a class from the provided ontology is mentioned in the input text. If the class is not known, output "None".\n2. Assign an ID to each instance, using the format "Class" + "1", where "Class" is the name of the detected class  + is concatenation. Use the instance\'s ID as the subject in each extracted triple.\n3. Create a triple between the ID and the class type using the rdf:type relationship.\n4. Identify any relationships mentioned in the input text regarding the detected class. Search the ontology for triples of the form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship for the detected class. Also, search for triples of the form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes.\n5. Extract the possible mentioned object in the input text, regarding the detected instance, for each detected relationship.\n6. Place each triple in a JSON object, in the format: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond with the JSON object(s) in a list. If no triple is detected, output "None".\n Provided ontology: {ontology} \n'},
  #            {4.1: f'You are a Knowledge Graph Expert. You are given a domain ontology in Turtle syntax, delimited by double quotes. Your task is to process a natural language text input, extracting triples about instances mentioned in the text based on the provided ontology. The input text may contain references to instances of classes in the ontology, along with specific relationships. Your task is to:\n\n1. Identify if a class from the provided ontology is mentioned in the input text. If the class is not known, output "None".\n2. Assign an ID to each instance, using the format "Class" + "1", where "Class" is the name of the detected class + is concatenation. Use the instance\'s ID as the subject in each extracted triple.\n3. Create a triple between the ID and the class type using the rdf:type relationship.\n4. Identify any relationships mentioned in the input text regarding the detected class. Search the ontology for triples of the form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship for the detected class. Also, search for triples of the form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes.\n5. Extract the possible mentioned object in the input text, regarding the detected instance, for each detected relationship.\n6. Place each triple in a JSON object, in the format: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond with the JSON object(s) in a list. If no triple is detected, output "None".\n Provided ontology: {ontology} \n'},
  #            {4.2: f'You are a Knowledge Graph Expert. You are given a domain ontology in Turtle syntax, delimited by double quotes. Your task is to process a natural language text input, extracting triples about instances mentioned in the text based on the provided ontology. The input text may contain references to instances of classes in the ontology, along with specific relationships. Your task is to:\n1. Identify if a class from the provided ontology is mentioned in the input text. If the class is not known, output "None".\n2. Assign an ID to each instance, using the format "Class" + "1", where "Class" is the name of the detected class + is concatenation. Use the instance\'s ID as the subject in each extracted triple.\n3. Create a triple between the ID and the class type using the rdf:type relationship.\n4. Identify any relationships mentioned in the input text regarding the detected class. Search the ontology for triples of the form :RelationshipName rdfs:domain :ClassName, and take the relationship name as a known relationship for the detected class. Also, search for triples of the form :Relationship rdfs:domain owl:Thing, as owl:Thing can be used as the superclass of all classes.\n5. Extract the possible mentioned object in the input text, regarding the detected instance, for each detected relationship.\n6. Place each triple in a JSON object, in the format: {{"subject" : ID, "relationship" : value, "object" : value}}. If any triple refers to another instance, add all triples you assumed of that instance too. Respond with the JSON object(s) in a list. If no triple is detected, output "None".\n Provided ontology: {ontology} \n'},
  #          ]
  #      },
    'gpt-3.5-turbo':
     {1.0:
        [
            {1: f'Given a domain ontology represented in Turtle syntax and enclosed with double quotes, along with a natural language input text, as a Knowledge Graph Expert, your task is to extract triples related to instances mentioned in the input. Each instance should be represented with an ID in the format "Class" + "1", where "Class" is the name of the detected class type and + is concatenation. Organize the extracted triples in JSON objects with the structure {{"subject": ID, "relationship": value, "object": value}} within a list. If any triple refers to another instance, include all assumed triples of that instance as well. Output only the list. In case no triples are found, output "None".\n Provided ontology: {ontology} \n'},
            {2: f'Given a domain ontology in Turtle syntax, enclosed with double quotes, and a natural language text input, as a Knowledge Graph Expert, your task is to identify instances of classes mentioned in the text and extract triples about them. Each instance should be represented by an ID in the format "Class" + "1", where "Class" is the name of the detected class type and + is concatenation, and linked to its class type using the rdf:type relationship. Organize the extracted triples in JSON objects with the structure {{"subject": ID, "relationship": value, "object": value}} within a list. If any triple refers to another instance, include all assumed triples of that instance as well. Output only the list. If no triples are found, output "None".\n Provided ontology: {ontology} \n'},
            {3: f'Given a domain ontology presented in Turtle syntax and enclosed with double quotes, your task as a Knowledge Graph Expert is to analyze a natural language text. Extract triples related to instances of classes mentioned in the input text by following these steps: \n1. Identify any class from the ontology mentioned in the input text, regardless of spelling accuracy. \n2. Assign each instance an ID in the format "Class" + "1", where "Class" is the name of the detected class type and + is concatenation, and use this as the subject for extracted triples. \n3. Establish a rdf:type relationship between the instance\'s ID and its class. \n4. Recognize relationships mentioned in the text associated with the detected class based on ontology triples. \n5. Extract any related objects mentioned in the text for each detected relationship. \n6. Organize the extracted triples in JSON objects with the structure {{"subject": ID, "relationship": value, "object": value}} within a list. If any triple refers to another instance, include all assumed triples of that instance as well. Output only the JSON object(s) in a list format. If no triples are found, output "None"."\n Provided ontology: {ontology} \n'},
            {4.1: f'Given a domain ontology presented in Turtle syntax and enclosed with double quotes, your task as a Knowledge Graph Expert is to analyze a natural language text. Extract triples related to instances of classes mentioned in the input text by following these steps: \n1. Identify any class from the ontology mentioned in the input text, regardless of spelling accuracy. \n2. Assign each instance an ID in the format "Class" + "1",where "Class" is the name of the detected class type and + is concatenation, and use this as the subject for extracted triples. \n3. Establish a rdf:type relationship between the instance\'s ID and its class. \n4. Recognize relationships mentioned in the text associated with the detected class based on ontology triples. \n5. Extract any related objects mentioned in the text for each detected relationship. \n6. Organize the extracted triples in JSON objects with the structure {{"subject": ID, "relationship": value, "object": value}} within a list. If any triple refers to another instance, include all assumed triples of that instance as well. Output only the JSON object(s) in a list format. If no triples are found, output "None"."\n Provided ontology: {ontology} \n'},
            {4.2: f'Given a domain ontology presented in Turtle syntax and enclosed with double quotes, your task as a Knowledge Graph Expert is to analyze a natural language text. Extract triples related to instances of classes mentioned in the input text by following these steps: \n1. Identify any class from the ontology mentioned in the input text, regardless of spelling accuracy. \n2. Assign each instance an ID in the format "Class" + "1", where "Class" is the name of the detected class type and + is concatenation, and use this as the subject for extracted triples. \n3. Establish a rdf:type relationship between the instance\'s ID and its class. \n4. Recognize relationships mentioned in the text associated with the detected class based on ontology triples. \n5. Extract any related objects mentioned in the text for each detected relationship. \n6. Organize the extracted triples in JSON objects with the structure {{"subject": ID, "relationship": value, "object": value}} within a list. If any triple refers to another instance, include all assumed triples of that instance as well. Output only the JSON object(s) in a list format. If no triples are found, output "None"."\n Provided ontology: {ontology} \n'},
            ]
      },
        'gpt-4o':
      {1.0:
        [
            {1: f'Imagine you are an expert in Knowledge Graphs. You are given a specific ontology in Turtle syntax, and it is enclosed in double quotes. The task involves processing a natural language text to extract data in the form of triples, which refer to the given classes and relationships in the ontology. Each identified instance should be denoted by a unique ID in the format "Class" + "1", where "Class" is the class name and + denotes concatenation. The extracted triples must be formatted as JSON objects like this: {{"subject" : ID, "relationship" : value, "object" : value}}. If a triple involves another instance, include all related triples. Only provide the JSON objects in a list as the output. If no triples are found, respond with "None".\n Do not add your JSON tags at the beginning (ex. "```json") and end of the output text (ex. "```"), leave only your answer.\n Here is the provided ontology: {ontology} \n'},
            {2: f'Your role is a Knowledge Graph Expert working with a domain ontology described using Turtle syntax. The ontology is provided within double quotes. You will receive a natural language text, which may or may not mention instances of the classes defined in the ontology along with specific relationships. Your task is to use the provided ontology to extract RDF triples about the mentioned instances from the input text. Each detected instance should be given an ID in the format: "Class" + "1", where "Class" is the name of the detected class. You must always create a triple that specifies the ID and its class type using the rdf:type relationship. Present each triple as a JSON object in the following structure: {{"subject" : ID, "relationship" : value, "object" : value}}. If a triple mentions another instance, include all triples related to that instance too. Provide your response as a list of JSON objects. If no triples are detected, output "None".\n Do not add your JSON tags at the beginning (ex. "```json") and end of the output text (ex. "```"), leave only your answer.\n Provided ontology:  {ontology} \n'},
            {3: f'You are an expert in Knowledge Graphs. You have been given a domain ontology within double quotes, described using the Turtle syntax. Your task is to analyze a natural language input text, which may or may not refer to instances of classes and relationships described in the ontology. Using the ontology provided, your job is to extract triples about the instances mentioned in the input text as follows: \n1. Determine if any class from the ontology is mentioned in the input text. Look in the ontology for triples like :ClassName rdf:type owl:Class, where ClassName is the class name. The class name in the input text could be misspelled. If no known class is mentioned, output "None". \n2. Assign an ID to each identified instance using "Class" + "1", where "Class" is the detected class name. \n3. Always create a triple between the ID and the class type using the rdf:type relationship. \n4. Identify any mentioned relationships from the input text regarding the identified class. Search for triples like :RelationshipName rdfs:domain :ClassName or :Relationship rdfs:domain owl:Thing in the ontology, where owl:Thing represents the superclass of all classes. \n5. For each detected relationship, extract the mentioned object in the input text with respect to the identified instance. \n6. Present each triple as a JSON object with the format: {{"subject" : ID, "relationship" : value, "object" : value}}. If a triple refers to another instance, include all triples for that instance as well. If no triples are detected, output "None".\n Do not add your JSON tags at the beginning (ex. "```json") and end of the output text (ex. "```"), leave only your answer.\n Provided ontology: {ontology} \n'},
            {4.1: f'You are an expert in Knowledge Graphs. You have been given a domain ontology within double quotes, described using the Turtle syntax. Your task is to analyze a natural language input text, which may or may not refer to instances of classes and relationships described in the ontology. Using the ontology provided, your job is to extract triples about the instances mentioned in the input text as follows: \n1. Determine if any class from the ontology is mentioned in the input text. Look in the ontology for triples like :ClassName rdf:type owl:Class, where ClassName is the class name. The class name in the input text could be misspelled. If no known class is mentioned, output "None". \n2. Assign an ID to each identified instance using "Class" + "1", where "Class" is the detected class name. \n3. Always create a triple between the ID and the class type using the rdf:type relationship. \n4. Identify any mentioned relationships from the input text regarding the identified class. Search for triples like :RelationshipName rdfs:domain :ClassName or :Relationship rdfs:domain owl:Thing in the ontology, where owl:Thing represents the superclass of all classes. \n5. For each detected relationship, extract the mentioned object in the input text with respect to the identified instance. \n6. Present each triple as a JSON object with the format: {{"subject" : ID, "relationship" : value, "object" : value}}. If a triple refers to another instance, include all triples for that instance as well. If no triples are detected, output "None".\n Do not add your JSON tags at the beginning (ex. "```json") and end of the output text (ex. "```"), leave only your answer.\n Provided ontology: {ontology} \n'},
            {4.2: f'You are an expert in Knowledge Graphs. You have been given a domain ontology within double quotes, described using the Turtle syntax. Your task is to analyze a natural language input text, which may or may not refer to instances of classes and relationships described in the ontology. Using the ontology provided, your job is to extract triples about the instances mentioned in the input text as follows: \n1. Determine if any class from the ontology is mentioned in the input text. Look in the ontology for triples like :ClassName rdf:type owl:Class, where ClassName is the class name. The class name in the input text could be misspelled. If no known class is mentioned, output "None". \n2. Assign an ID to each identified instance using "Class" + "1", where "Class" is the detected class name. \n3. Always create a triple between the ID and the class type using the rdf:type relationship. \n4. Identify any mentioned relationships from the input text regarding the identified class. Search for triples like :RelationshipName rdfs:domain :ClassName or :Relationship rdfs:domain owl:Thing in the ontology, where owl:Thing represents the superclass of all classes. \n5. For each detected relationship, extract the mentioned object in the input text with respect to the identified instance. \n6. Present each triple as a JSON object with the format: {{"subject" : ID, "relationship" : value, "object" : value}}. If a triple refers to another instance, include all triples for that instance as well. If no triples are detected, output "None".\n Do not add your JSON tags at the beginning (ex. "```json") and end of the output text (ex. "```"), leave only your answer.\n Provided ontology: {ontology} \n'}
            ]
      }
    }

# dummy dataset
'''dataset = [
            {'text':'insert a project with code as something like 4A7 manager is someone with anything assistant class is Java status is something with anything finished and name is BestRobot.', 'golden_labels' : '[{"subject" : "Project1", "relationship" : "rdf:type" , "object" : "Project" }, {"subject" : "Project1", "relationship" : "hasCode" , "object" : "4A7" }, {"subject" : "Project1", "relationship" : "hasClass" , "object" : "Java" }, {"subject" : "Project1", "relationship" : "hasManager", "object" : "Employee1" }, {"subject" : "Employee1", "relationship" : "hasRole", "object" : "assistant" }, {"subject" : "Employee1", "relationship" : "rdf:type", "object" : "Employee" }, {"subject" : "Project1", "relationship" : "hasStatus" , "object" : "Status1" }, {"subject" : "Status1", "relationship" : "rdf:type" , "object" : "Status" },{"subject" : "Status1", "relationship" : "hasName" , "object" : "finished" }, {"subject" : "Project1", "relationship" : "hasName" , "object" : "BestRobot" }]','alternative_labels' : '[{"subject" : "Project1", "relationship" : "hasStatus" , "object" : "finished" }]', 'fp_ok_labels':'[]', 'text_type' : 'Project', 'class_type' : 'S'},
            {'text':'insert a project with code WF6, name as BestRobot, class is java and Employee123 as manager.', 'golden_labels' : '[{"subject" : "Project1", "relationship" : "rdf:type" , "object" : "Project" }, {"subject" : "Project1", "relationship" : "hasCode" , "object" : "WF6" }, {"subject" : "Project1", "relationship" : "hasClass" , "object" : "java" }, {"subject" : "Project1", "relationship" : "hasManager", "object" : "Employee123" }, {"subject" : "Employee123", "relationship" : "rdf:type", "object" : "Employee" }, {"subject" : "Project1", "relationship" : "hasName" , "object" : "BestRobot" }]', 'alternative_labels' : '[]', 'fp_ok_labels':'[{"subject" : "Employee123", "relationship" : "hasRole", "object" : "Manager"}, {"subject" : "Employee123", "relationship" : "hasRole", "object" : "manager"}]', 'text_type' : 'Project', 'class_type' : 'S'},
            {'text':'Tell me more about the reports we have in the database.','golden_labels':'None', 'alternative_labels' : '[]', 'fp_ok_labels' : '[]', 'text_type' : 'None', 'class_type' : 'SN'}, # None output
            {'text':'Please add a prject with code as 123 and manager as John','golden_labels':'[{"subject" : "Project1", "relationship" : "rdf:type" , "object" : "Project" }, {"subject" : "Project1", "relationship" : "hasCode" , "object" : "123" }, {"subject" : "Project1", "relationship" : "hasManager", "object" : "Employee1" }, {"subject" : "Employee1", "relationship" : "hasName", "object" : "John" }]', 'alternative_labels' : '[{"subject" : "Project1", "relationship" : "hasManager", "object" : "John"}]', 'fp_ok_labels':'[{"subject" : "Employee1", "relationship" : "hasRole", "object" : "Manager"}, {"subject" : "Employee1", "relationship" : "hasRole", "object" : "manager"}]'}, # misspelled type
            {'text':'i want you to insert an app instance with code being something like A-9I its class is Pascal named BestApp and put Robert as the manager','golden_labels':'None', 'alternative_labels' : '[]', 'fp_ok_labels' : '[]'} # not known type
           ]'''

In [52]:
# set your own preferences

sys_mesg_ord =  0 # the position of the system message regarding the prompt; 0 - beginning, 1 - end
file = 'all_prompts_sports_sys_mesg_ord_0.txt' # file path to write the serialized prompts
download_files = False # option to download the files

In [ ]:
all_prompts = generate_prompts(system_prompts, dataset, versions = [1], sys_mesg_ord =  sys_mesg_ord, file = file, download = download_files)

## Load prompts (enter to set variables)

In [ ]:
# set your own preferences

file = 'run_3_all_prompts_sports_sys_mesg_ord_0.txt' # file path to load from the serialized prompts

In [ ]:
all_prompts = from_text_to_Prompt(file)

## General Pipeline (enter to set variables)

In [54]:
# set your own preferences

file = 'run_1_2_3_all_prompts_sports_sys_mesg_ord_0_GPT.txt' # path where to save the serialized predicted Prompt objects
download_files = False # option to download the serialized predicted Prompt objects

In [ ]:
# !!! ONLY RUN THIS CELL IF YOU INTEND TO CALL MIXTRAL-8x7b OR ANY OTHER MODEL FROM HUGGING FACE

bnb4_config =  BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4', # precision of the stored weights
                              bnb_4bit_compute_dtype='bfloat16', # precision of computations
                              bnb_4bit_use_double_quant=True
                              )

                # set your own quantization_config if desired

model = AutoModelForCausalLM.from_pretrained('mistralai/Mixtral-8x7B-Instruct-v0.1',
                                                         trust_remote_code = True,
                                                         quantization_config = bnb4_config
                                                         ) # for training, device_map does not have to be set. check -> https://huggingface.co/docs/transformers/v4.35.0/main_classes/quantization#bitsandbytes-integration


In [ ]:
%%time

# provide the models' checkpoints, optional their max context window
# !!! EITHER CALL query FOR API CALLS TO ALL MODELS, OR query2 TO PROMPT HUGGING FACE MODELS THAT WERE LOCALLY LOADED
checkpoint = [
              {
                "model" : "mistralai/Mixtral-8x7B-Instruct-v0.1",  # doesn't have system prompt
                "max_context_window" : 32768
              },
              #{
              #  "model" : "mistralai/Mistral-7B-Instruct-v0.1", # doesn't have system prompt
              #  "max_context_window" : 32768
              #},
              #{
              #  "model" : "mistralai/Mistral-7B-Instruct-v0.2", # doesn't have system prompt
              #  "max_context_window" : 32768
              #},
              #{
              #  "model" : "openchat/openchat-3.5-0106", # doesn't have system prompt
              #  "max_context_window" : 8192
              #},
              #{
              #  "model" : "HuggingFaceH4/zephyr-7b-beta", # does have system prompt
              #  "max_context_window"  : 32768
              #},
              #{
              #  "model" : 'gpt-3.5-turbo-0125', # does have system prompt
              #  "max_context_window" : 16385
              #},
              {
                "model" : 'gpt-4o', # does have system prompt
                "max_context_window" : 128000
              },
              {
                "model" : 'gpt-4.1-2025-04-14', # does have system prompt
                "max_context_window" : 128000
              }
              ]

#predictions = query(checkpoint, all_prompts, file, download = download_files)
predictions = query2(checkpoint, all_prompts, model, False, download = download_files)

In [ ]:
# view the predictions

view_predictions(predictions, all_prompts, False)

In [ ]:
# generate the metrics reports

# set your own preferences

# as input, the user has to specify what combination of levels they want their metrics to be calculated at, from:
# 0 - at model level, 1 - at prompt's level, 2 - at prompt's version, 3 - at the class type (Project, Employee, Status, None), 4 - at input text's type
# eg. providing [0], will have the metrics for each model's performance, [1] gives the performance of all models at a specific prompt level, while [0, 1] will give the permformance of each model at a specific prompt's level


levels = [0]
file = "results_run_2_3_all_prompts_sports_sys_mesg_ord_0.txt"
template_strict = True
triple_strict = True
download_files = False

view_metrics(predictions, file, levels, template_strict = template_strict, triple_strict = triple_strict, download = download_files)

## Calculate metrics from existing predictions (enter to set variables)

In [ ]:
# set your own preferences
from google.colab import files


file = 'run_1_2_3_all_prompts_sports_sys_mesg_ord_0_GPT.txt'

#uploaded = files.upload()

predictions2 = from_text_to_Prompt(file)

In [ ]:
# view the predictions

view_predictions(predictions2, all_prompts, False)

In [ ]:
# generate the metrics reports

# set your own preferences

# as input, the user has to specify what combination of levels they want their metrics to be calculated at, from:
# 0 - at model level, 1 - at prompt's level, 2 - at prompt's version, 3 - at the class type (Project, Employee, Status, None), 4 - at input text's type
# eg. providing [0], will have the metrics for each model's performance, [1] gives the performance of all models at a specific prompt level, while [0, 1] will give the permformance of each model at a specific prompt's level

levels = [0,3]
file = "results_run_1_2_3_all_prompts_sports_sys_mesg_ord_0_GPT.txt"
template_strict = True
triple_strict = True
download_files = False

view_metrics(predictions2, file, levels, template_strict = template_strict, triple_strict = triple_strict, download = download_files)